### Instalar librerias y dependencias necesarias para el proyecto

In [1]:
!pip install python-dotenv --upgrade
!pip install boto3 --upgrade
!pip install pyaml --upgrade
!pip install Jinja2 --upgrade
!pip install opensearch-py
!pip install tqdm
!pip install beautifulsoup4
!pip install fastparquet

  Attempting uninstall: python-dotenv
    Found existing installation: python-dotenv 1.0.0
    Uninstalling python-dotenv-1.0.0:
      Successfully uninstalled python-dotenv-1.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 52.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 2.0 MB/s eta 0:00:00:00:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.131
    Uninstalling botocore-1.34.131:
      Successfully uninstalled botocore-1.34.131
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.6.2
    Uninstalling s3transfer-0.6.2:
      Successfully uninstalled s3transfer-0.6.2
  Attempting uninstall: boto3
    Found existing installation: boto3 1.26.100
    Uninstalling boto3-1.26.100:
      Successfully uninstalled boto3-1.26.100
ERROR: pip's dependency resolver does not currently take into a

### Importar librerias y dependencias

In [1]:
import os
import sys
import gc
import logging
import boto3
import botocore
import yaml
import json
import requests
import zipfile
import pandas as pd
import numpy as np
import fastparquet
import re

from IPython.display import FileLink
from bs4 import BeautifulSoup
from tqdm import tqdm
from typing import Callable
from time import time, sleep

from jinja2 import Template, Environment
from dotenv import load_dotenv, find_dotenv
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from opensearchpy.helpers import bulk, streaming_bulk

# Configurar el logging
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    handlers=[
                        logging.FileHandler("drugs.log"),
                        logging.StreamHandler()
                    ]
                   )

### Cargar variables de entorno y archivos de configuración

In [2]:
# Carga variables entorno y archivos de configuración
load_dotenv(find_dotenv())
        
# Cargar archivo de configuraciones .yaml
yaml_data = None
try:
    # Definir una función para cargar variables de entorno
    def get_env_variable(name):
        return os.getenv(name)

    # Crear un entorno Jinja2 con la función de carga de variables de entorno
    env = Environment()

    # Agregar la función de entorno al entorno Jinja2
    env.globals['env'] = get_env_variable

    with open('config.yaml', 'r') as file:
        template_content = file.read()
    
     # Cargar el contexto YAML
    yaml_data = yaml.safe_load(template_content)
    
     # Renderizar la plantilla con el contexto
    template = env.from_string(template_content)
    rendered_content = template.render(**yaml_data)
    
    # Cargar el YAML renderizado
    yaml_data = yaml.safe_load(rendered_content)
    
    print("Archivo de configuracion cargado exitosamente.")
except FileNotFoundError:
    print("Error: El archivo de configuración .yaml no se encuentra disponible")
except yaml.YAMLError as exc:
    print(f"Error al analizar el archivo YAML: {exc}")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

Archivo de configuracion cargado exitosamente.


In [7]:
#Valida que se hayan definido las variables de configuración y las variables de entorno
try:
    
    if not(yaml_data):
        sys.exit('No se puede continuar porque no se han definido el archivo .yaml del Proyecto.')
    
    # Cargar variables de entorno
    aws_region = os.environ.get('aws_region')
    aws_access_key_id = os.environ.get('aws_access_key_id')
    aws_secret_access_key = os.environ.get('aws_secret_access_key')
    
    # Cargar variables de configuración
    project = yaml_data.get('Project', {})
    if not(project):
        sys.exit('No se puede continuar porque no se han definido la configuración del Proyecto.')
    
    project_name = project.get('name') if(project is not None and project.get('name')) else None
    url_dataset = project.get('url_dataset') if(project is not None and project.get('url_dataset')) else None
    chunk_size = project.get('chunk_size') if(project is not None and project.get('chunk_size')) else None
    file_name_data_parquet = project.get('file_name_data_parquet') if(project is not None and project.get('file_name_data_parquet')) else None
    open_fda_columns = yaml_data.get('OpenFDA',{}) if(yaml_data is not None and yaml_data.get('OpenFDA',{})) else None
    
    
    open_search = yaml_data.get('OpenSearch', {}) if(yaml_data.get('OpenSearch', {}) is not None) else None
    service = open_search.get('service') if(open_search is not None and open_search.get('service')) else None
    port = open_search.get('port') if(open_search is not None and open_search.get('port')) else None
    use_ssl = open_search.get('use_ssl') if(open_search is not None and open_search.get('use_ssl')) else None
    verify_certs = open_search.get('verify_certs') if(open_search is not None and open_search.get('verify_certs')) else None
    pool_maxsize = open_search.get('pool_maxsize') if(open_search is not None and open_search.get('pool_maxsize')) else None
    timeout = open_search.get('timeout') if(open_search is not None and open_search.get('timeout')) else None
    collection = open_search.get('Collection',{}) if(open_search is not None and open_search.get('Collection')) else None
    collection_name = collection.get('name') if(collection is not None and collection.get('name')) else None
    collection_index = collection.get('index') if(collection is not None and collection.get('index')) else None
    
    
    '''
    if(aws_region is None):
        raise Exception('No se ha definido la variable de entorno: aws_region')
    elif(aws_access_key_id is None):
        raise Exception('No se ha definido la variable de entorno: aws_access_key_id')
    elif(aws_secret_access_key is None):
        raise Exception('No se ha definido la variable de entorno: aws_secret_access_key')
    '''
    
    if(project_name is None):
        raise Exception('No se ha definido la variable de configuración del proyecto : project_name')
    elif(url_dataset is None):
        raise Exception('No se ha definido la variable de configuración del proyecto : url_dataset')
    elif(chunk_size is None):
        raise Exception('No se ha definido la variable de configuración del proyecto : chunk_size')
    elif(file_name_data_parquet is None):
        raise Exception('No se ha definido la variable de configuración del proyecto : file_name_data_parquet')
    elif(open_fda_columns is None):
        raise Exception('No se ha definido la variable de configuración del proyecto : open_fda_columns')
    
    
    '''
    if(service is None):
        raise Exception('No se ha definido la variable de configuración de openSearch-AWS : service')
    elif(port is None):
        raise Exception('No se ha definido la variable de configuración de openSearch-AWS : port')
    elif(use_ssl is None):
        raise Exception('No se ha definido la variable de configuración de openSearch-AWS : use_ssl')
    elif(verify_certs is None):
        raise Exception('No se ha definido la variable de configuración de openSearch-AWS : verify_certs')
    elif(pool_maxsize is None):
        raise Exception('No se ha definido la variable de configuración de openSearch-AWS : pool_maxsize')
    elif(timeout is None):
        raise Exception('No se ha definido la variable de configuración de openSearch-AWS : timeout')
    elif(collection_name is None):
        raise Exception('No se ha definido la variable de configuración de openSearch-AWS : collection_name')
    elif(collection_index is None):
        raise Exception('No se ha definido la variable de configuración de openSearch-AWS : collection_index')
    '''
    
    # Define variables de entorno y variables globales
    DATASET_FOLDER = project_name + '/dataset'
    
    # Define la ruta y el nombre para almacenar y cargar el archivo parquet
    DATA_OUTPUT_PARQUET_FILE = DATASET_FOLDER + '/' + file_name_data_parquet
    
    
     # Crea directorio para almacenar los datasets
    if not(os.path.exists(DATASET_FOLDER)):
        !mkdir -p {DATASET_FOLDER}
        print('Directorio creado exitosamente!!')
    
except Exception as ex:
    print(ex)
    print(f'Error en la linea No. {ex.__traceback__.tb_lineno}')

### Definición de Funciones

In [23]:
#  Decorador para medir el tiempo de ejecución de la función
def measure_time(func:Callable)->Callable:
    '''
    args: one argument, is the function that we want to decorated(e,g functions that:
    build the model, train the model, optimize the model, ...etc)
    '''
    # define the wrapper function that executes the original function(e,g arg func)
    def wrapper(*args, **kwargs)->float:
        '''
        the wrapper function have the same argument as the object(e,g arg)function
        '''
        start_time=time()
        # call the object function to process and manipulate ...
        result=func(*args, **kwargs)
        end_time=time()

        processing_time=end_time-start_time
        print(f'tiempo de procesamiento {processing_time:.2f} segundos')
        return result

    return wrapper


# Función para validar si una URL es válida
def is_url_valid(url : str) -> bool:
    try:
        response = requests.head(url)
        return response.status_code == 200
    except Exception:
        return False


# Función para descargar archivos desde repositorio openFDA
@measure_time
def download_files(url_file : str, chunk_size: int = 1024):
    try:
        # Hacer la solicitud para descargar el archivo .zip
        response = requests.get(url_file, stream=True)
        if(response.status_code != 200):
            raise Exception(f"Error en la solicitud HTTP: {url_file}| respuesta: {response.status_code}")
        
        zip_filename = os.path.join(DATASET_FOLDER, url_file.split('/')[-1])
        
        total_size = int(response.headers.get('content-length', 0))
        
        # Guardar el archivo .zip descargado
        with open(zip_filename, 'wb') as file:
            file.write(response.content)
            #for data in tqdm(response.iter_content(chunk_size=chunk_size), total=total_size//chunk_size, unit='KB'):
            #    file.write(data)
                    
        logging.debug("Archivo guardado como: {}".format(zip_filename))
        
        # Extraer el contenido del archivo .zip
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall(DATASET_FOLDER)
        logging.debug("Archivo extraído: {}".format(zip_filename))

        # Eliminar el archivo .zip descargado
        os.remove(zip_filename)
        logging.debug("Archivo .zip eliminado {}".format(zip_filename))
        
    except zipfile.BadZipFile:
        logging.error("El archivo no es un archivo zip válido: {}".format(zip_filename))
    except FileNotFoundError:
        logging.error("El archivo no fue encontrado: {}".format(zip_filename))
    except Exception as ex:
        logging.error("Ocurrió un error al extraer el archivo {}: {}".format(zip_filename, e))
        

# Funcion para establecer conexion con OpenSearch-Serverless de AWS 
def get_connection_aws():
    try:
        
        # Establecer conexión con OpenSearch-AWS-Serverless
        session = boto3.Session(aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=aws_region)
        client = session.client('opensearchserverless')
        credentials = session.get_credentials()
        awsauth = AWSV4SignerAuth(credentials, aws_region, service)  
        response = client.batch_get_collection(names=[collection_name])

        #sleep(2)

        # Extraer el endpoint de la colección desde el response
        aws_host = (response['collectionDetails'][0]['collectionEndpoint'])
        aws_host = aws_host.replace("https://", "")

        # Construir el cliente de OpenSearch
        client = OpenSearch(hosts=[{'host': aws_host, 'port': port}],
                            http_auth=awsauth,
                            use_ssl=use_ssl,
                            verify_certs=verify_certs,
                            connection_class=RequestsHttpConnection,
                            timeout=timeout,
                            pool_maxize=pool_maxsize,
                            max_retries=3,  # Número máximo de reintentos
                            retry_on_timeout=True  # Reintentar en caso de timeout
                            )
        return client
    except Exception as ex:
        logging.error("Ocurrió un error al conectar AWS {} ".format(ex))
        print(f"Ocurrió un error al conectar AWS {ex}")
        print(ex.__traceback__.tb_lineno)

# Función para enviar de forma masiva una colección de objetos a OpenSearch de AWS 
def streaming_bulk_collect(batch):
    try:
        responses = []
        success = 0
        failed = 0
        
        for ok, response in streaming_bulk(client=client,
                                        actions=batch,
                                        chunk_size = chunk_size,  # Número de documentos por cada chunk
                                        max_chunk_bytes=150 * 1024 * 1024,  # 150 MB
                                        raise_on_error=False,
                                        raise_on_exception=False,
                                        max_retries=3, # No. máximo de veces que el cliente intentará reintentar una operación fallida
                                        initial_backoff=2, # Especifica el tiempo de espera en segundos antes de realizar el primer reintento
                                        max_backoff=5, # Especifica el tiempo máximo de espera en segundos entre reintentos
                                        yield_ok=True,
                                        ignore_status=[409]):
            if ok:
                success += 1
            else:
                failed += 1
                error = response['index']['_id'] + ":" + response['index'].get('error')
                responses.append(error)
        return success, failed, responses

    except Exception as ex:
        logging.error("Ocurrió un error al procesar los datos {} ".format(ex))
        print(f"Ocurrió un error al procesar los datos {ex}")
        print(ex.__traceback__.tb_lineno)


# Funcion para limpiar los datos y quitar caracteres innesarios como los html
def remove_html_tags(text):
    sopa = BeautifulSoup(text, 'html.parser')
    text = sopa.get_text()
    return text


clean = re.compile('<.*?>')

# Funcion para limpiar los datos y quitar caracteres innesarios como los html
def remove_tags_html(text):
    #text = text.decode('utf-8')
    return re.sub(clean, '', text)


# Funcion para convertir un objeto tipo lista a un objeto tipo string  
def safe_encode(text):
    if(text is not None):
        if isinstance(text, list):
            #if pd.notna(text).all() and pd.notnull(text).all() and text != '':
            text = " ".join(map(str, text))
        else:
            text = str(text)
    return text
    #return str(text).encode('utf-8')


# Función para reemplazar 'nan' por una cadena vacia
def replace_nan_with_empty_string(example):
    text = {
        key : '' if(pd.isna(value) or (value == 'nan')) else value for key, value in example.items()
    }
    return text

 
# Funcion para obtener los documentos de OpenFDA descargados en formato json
def get_data_drugs_file(json_file):
    try:
        if(json_file):
            data = json.load(json_file)
            last_updated = data['meta']['last_updated']
            batch = []
            
            i= 0
            for item in data['results']:
                #if(i == 0):
                item['last_updated'] = last_updated

                # Crear un nuevo diccionario con la estructura deseada y fusionar openfda si existe
                result = {**{k: v for k, v in item.items() if k != "openfda"}, **item.get("openfda", {})}

                #if(result['id']== '1e91605f-fccf-f36e-e063-6394a90ab8f9'):
                '''
                # Paso 1: Convertir listas en cadenas de texto con elementos concatenados por un salto de línea
                result = {
                    key: " ".join(map(str, value)) if isinstance(value, list) else value
                    for key, value in result.items()
                }
                
                
                # Paso 2: Eliminar etiquetas HTML de las cadenas de texto
                result = {
                    key: remove_tags_html(value) if isinstance(value, str) else value
                    for key, value in result.items()
                }
                '''
                
                #print(json.dumps(result, indent=2))

                batch.append(result)

                # Si el tamaño del lote es 500, hacer yield y reiniciar el lote para optimizar el uso de la memoria
                if len(batch) == chunk_size:
                    yield batch
                    batch = []
                
                i+=1
                
                
                '''
                action = {
                    "_index": collection_index,
                    "_id": results['id'],  # Usar el atributo 'id' como ID del documento
                    "_source": results
                }
                '''
                
            if batch is not None:
                yield batch
            
    except Exception as ex:
        logging.error("Ocurrió un error al leer los datos json: {} ".format(ex))
        print(f"Ocurrió un error al leer los datos json: {ex}")
        print(ex.__traceback__.tb_lineno)


# Funcion para cargar el dataset de medicamentos almacenados en los archivos json de openFDA
def load_dataset_drug(load_type: str = 'columns'):
    try:
        #get_connection_aws()
        
        # Cargar la estructura de columnas de OpenFDA para obtener la base del dataset descargado
        if(open_fda_columns is not None):
            df_open_fda = pd.DataFrame(columns=open_fda_columns)
            
            # Leer archivos JSON
            json_files = [pos_json for pos_json in os.listdir(DATASET_FOLDER) if pos_json.endswith('.json')]

            if(json_files is not None and len(json_files) > 0):
                json_files = sorted(json_files, key=lambda x: int(x.split('-')[2]))

                acu = 0
                
                # Leer el contenido de cada archivo JSON
                for json_file in json_files:
                    print(f'Load dataset : {json_file}')
                    with open(os.path.join(DATASET_FOLDER, json_file), 'r') as f:
                        for batch in get_data_drugs_file(f):
                            if(batch is not None and len(batch) > 0):
                            
                                # Calcular el tamaño del lote en KB
                                batch_size = len(json.dumps(batch)) / 1024 / 1204 # Convertir el lote a JSON y medir el tamaño en MB
                                print(f'Batch size: {batch_size:.2f} MB')
                                acu+= batch_size
                                
                                df_drugs_chunks = pd.json_normalize(batch)
                                if(df_drugs_chunks is not None and df_drugs_chunks.shape[0] > 0):
                                    
                                    #Enviar colección de datos de forma masiva usando bulk de OpenSearch.helper
                                    #print('===================================================================')
                                    #success, failed, responses = streaming_bulk_collect(batch_size)
                                    #print('success : ', success)
                                    #print('failed : ', failed)
                                    #print('===================================================================')

                                    #Inicia pre-procesamiento para limpiar y quitar datos innecesarios para el entrenamiento del modelo
                                    #==================================================================================================
                                    
                                    # Reindexar el nuevo DataFrame para que coincida con las columnas generales
                                    df_drugs_chunks = df_drugs_chunks.reindex(columns=df_open_fda.columns)
                                    
                                    # Concatenar los DataFrames
                                    df_drugs_result = pd.concat([df_open_fda, df_drugs_chunks], ignore_index=True)
                                    
                                    # Obtiene id drugs
                                    #df_drugs_result['id'][0]
                                    
                                    df_drugs_result = df_drugs_result.apply(lambda x: x.apply(safe_encode) if(x is not None) else x, axis=0)
                                    df_drugs_result = df_drugs_result.apply(lambda x: x.apply(remove_tags_html) if(x is not None) else x, axis=0)
                                    
                                    # Reemplazar NaN por una cadena vacía
                                    df_drugs_result = df_drugs_result.fillna('')
                                    
                                    # Reemplazar el string 'nan' con una cadena vacía
                                    df_drugs_result = df_drugs_result.replace('nan', '')
                                    
                                    try:
                                        if not os.path.exists(DATA_OUTPUT_PARQUET_FILE):
                                            # Si el archivo no existe, lo creamos
                                            fastparquet.write(DATA_OUTPUT_PARQUET_FILE, df_drugs_result, compression='SNAPPY')
                                        else:
                                            # Agregamos datos al archivo Parquet existente
                                            fastparquet.write(DATA_OUTPUT_PARQUET_FILE, df_drugs_result, append=True, compression='SNAPPY')
                                    except Exception as ex:
                                        print(f'Line No : {ex.__traceback__.tb_lineno} => {ex}')
                                        #print(batch)
                                    
                                    # Libera memoria
                                    del df_drugs_chunks
                                    del df_drugs_result
                                    gc.collect()
                                #break
                    #break
                print(f'Total Batch size: {acu:.2f} MB => {acu/1024:.2f} GB')
                
                    
    except Exception as ex:
        print(f"Ocurrió un error al procesar los archivos json : {ex}")
        print(ex.__traceback__.tb_lineno)

### Descargar dataset desde OpenFDA

In [12]:
# Descargar dataset desde OpenFDA
try:
    response = requests.get(url_dataset)
    if(response.status_code != 200):
        raise Exception(f"Error en la solicitud HTTP. Código de respuesta: {response.status_code}")
        
    data = response.json()
    
    # Obtener las particiones
    partitions = data.get('results', {}).get('drug', {}).get('label', {}).get('partitions', [])

    boo = False
    if isinstance(partitions, list):
        total_files = len(partitions)
        with tqdm(total=total_files, desc="Procesando descarga") as pbar:
            for partition in partitions:
                file_url = partition.get('file', '')
                if file_url and is_url_valid(file_url):
                    print(f"Descargando archivo desde: {file_url}")
                    download_files(file_url)
                    pbar.update(1)
                    #break
    else:
        print("El nodo 'partitions' no es una lista.")
        
except Exception as ex:
    print(ex)

Procesando descarga:   0%|          | 0/12 [00:00<?, ?it/s]

Descargando archivo desde: https://download.open.fda.gov/drug/label/drug-label-0001-of-0012.json.zip


Procesando descarga:   8%|▊         | 1/12 [00:10<01:54, 10.37s/it]

tiempo de procesamiento 9.83 segundos
Descargando archivo desde: https://download.open.fda.gov/drug/label/drug-label-0002-of-0012.json.zip


Procesando descarga:  17%|█▋        | 2/12 [00:22<01:52, 11.27s/it]

...

tiempo de procesamiento 8.33 segundos
Descargando archivo desde: https://download.open.fda.gov/drug/label/drug-label-0011-of-0012.json.zip


Procesando descarga:  92%|█████████▏| 11/12 [01:47<00:10, 10.40s/it]

tiempo de procesamiento 12.72 segundos
Descargando archivo desde: https://download.open.fda.gov/drug/label/drug-label-0012-of-0012.json.zip


Procesando descarga: 100%|██████████| 12/12 [01:51<00:00,  9.29s/it]

tiempo de procesamiento 4.15 segundos


### Procesar información del dataset descargado y realizar la limpieza de los datos

In [24]:
load_dataset_drug()

Load dataset : drug-label-0001-of-0012.json
Batch size: 55.98 MB
Batch size: 65.07 MB
Batch size: 66.60 MB
Batch size: 62.89 MB
Batch size: 33.89 MB
Batch size: 42.20 MB
Batch size: 70.76 MB
Batch size: 105.88 MB
Load dataset : drug-label-0002-of-0012.json
Batch size: 106.20 MB
Batch size: 70.59 MB
Batch size: 60.76 MB
Batch size: 66.22 MB
Batch size: 66.05 MB
Batch size: 52.92 MB
Batch size: 39.81 MB
Batch size: 46.59 MB
Load dataset : drug-label-0003-of-0012.json
Batch size: 106.50 MB
Batch size: 97.83 MB
Batch size: 78.22 MB
Batch size: 56.59 MB
Batch size: 66.10 MB
Batch size: 64.18 MB
Batch size: 64.91 MB
Batch size: 31.48 MB
Load dataset : drug-label-0004-of-0012.json
Batch size: 42.40 MB
Batch size: 93.85 MB
Batch size: 101.91 MB
Batch size: 83.68 MB
Batch size: 54.80 MB
Batch size: 60.38 MB
Batch size: 61.08 MB
Batch size: 61.95 MB
Load dataset : drug-label-0005-of-0012.json
Batch size: 28.45 MB
Batch size: 37.86 MB
Batch size: 96.53 MB
Batch size: 99.37 MB
Batch size: 94.25 MB

In [13]:
# Leer el contenido de cada archivo JSON
!ls -lrth $data_output_parquet_file
FileLink(DATA_OUTPUT_PARQUET_FILE)

-rwxrwxrwx 1 root root 177M ago 15 21:55 ./dataset/drugs_data.parquet


/media/ljama/DiscoLocalD/desarrollo/PythonProjects/drugs-generative/dataset/drugs_data.parquet

In [21]:
#!rm -R drugs-generative/dataset/*.parquet

-rw-r--r-- 1 root root 1.3G Aug  7 02:58 drugs-generative/dataset/drugs_data.parquet


In [11]:
df_drugs = pd.read_parquet(DATA_OUTPUT_PARQUET_FILE)

In [17]:
print(df_drugs.iloc[2345]['package_label_principal_display_panel'])

Package Labeling: Outer Package6 Inner Package6 Label TESTER Box


In [20]:
# Contar la cantidad de valores no vacíos (non-empty) por columna
non_empty_counts = df_drugs.apply(lambda x: x != '').sum()

# Crear un DataFrame con el resultado y ordenar
columns_info = non_empty_counts.reset_index()
columns_info.columns = ['Column', 'Non-Empty Count']
columns_info = columns_info.sort_values(by='Non-Empty Count', ascending=False)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(columns_info)

,Column,Non-Empty Count
44,last_updated,30000
34,id,30000
28,effective_time,30000
52,package_label_principal_display_panel,29957
36,indications_and_usage,28668
22,dosage_and_administration,28569
83,warnings,22825
2,active_ingredient,18399
35,inactive_ingredient,18355
31,generic_name,14320


In [ ]:
# Ejemplo de estructura json dataset openFDA

batch = {
      "spl_product_data_elements": [
        "Hydrocodone Bitartrate and Acetaminophen Hydrocodone Bitartrate and Acetaminophen HYDROCODONE BITARTRATE HYDROCODONE ACETAMINOPHEN ACETAMINOPHEN SILICON DIOXIDE CROSCARMELLOSE SODIUM CROSPOVIDONE MAGNESIUM STEARATE CELLULOSE, MICROCRYSTALLINE POVIDONE, UNSPECIFIED STARCH, CORN STEARIC ACID White to off-white U114 Hydrocodone Bitartrate and Acetaminophen Hydrocodone Bitartrate and Acetaminophen HYDROCODONE BITARTRATE HYDROCODONE ACETAMINOPHEN ACETAMINOPHEN SILICON DIOXIDE CROSCARMELLOSE SODIUM CROSPOVIDONE MAGNESIUM STEARATE CELLULOSE, MICROCRYSTALLINE POVIDONE, UNSPECIFIED STARCH, CORN STEARIC ACID White to off-white U115 Hydrocodone Bitartrate and Acetaminophen Hydrocodone Bitartrate and Acetaminophen HYDROCODONE BITARTRATE HYDROCODONE ACETAMINOPHEN ACETAMINOPHEN SILICON DIOXIDE CROSCARMELLOSE SODIUM CROSPOVIDONE MAGNESIUM STEARATE CELLULOSE, MICROCRYSTALLINE POVIDONE, UNSPECIFIED STARCH, CORN STEARIC ACID White to off-white U116"
      ],
      "boxed_warning": [
        "WARNING: ADDICTION, ABUSE, AND MISUSE;RISK EVALUATION AND MITIGATION STRATEGY (REMS) LIFE-THREATENING RESPIRATORY DEPRESSION; ACCIDENTAL INGESTION; NEONATAL OPIOID WITHDRAWAL SYNDROME; CYTOCHROME P450 3A4 INTERACTION; HEPATOTOXICITY; and RISKS FROM CONCOMITANT USE WITH BENZODIAZEPINES OR OTHER CNS DEPRESSANTS Addiction, Abuse, and Misuse Hydrocodone bitartrate and acetaminophen tablets expose patients and other users to the risks of opioid addiction, abuse, and misuse, which can lead to overdose and death. Assess each patient\u2019s risk prior to prescribing hydrocodone bitartrate and acetaminophen tablets, and monitor all patients regularly for the development of these behaviors and conditions [see WARNINGS ]. Opioid Analgesic Risk Evaluation and Mitigation Strategy (REMS): To ensure that the benefits of opioid analgesics outweigh the risks of addiction, abuse, and misuse, the Food and Drug Administration (FDA) has required a REMS for these products [see Warnings]. Under the requirements of the REMS, drug companies with approved opioid analgesic products must make REMS-compliant education programs available to healthcare providers. Healthcare providers are strongly encouraged to \u2022 complete a REMS-compliant education program, \u2022 counsel patients and/or their caregivers, with every prescription, on safe use, serious risks, storage, and disposal of these products, \u2022 emphasize to patients and their caregivers the importance of reading the Medication Guide every time it is provided by their pharmacist, and \u2022 consider other tools to improve patient, household, and community safety. Life-Threatening Respiratory Depression Serious, life-threatening, or fatal respiratory depression may occur with use of hydrocodone bitartrate and acetaminophen tablets. Monitor for respiratory depression especially during initiation of hydrocodone bitartrate and acetaminophen tablets or following a dos e increase [see WARNINGS ]. Accidental Ingestion Accidental ingestion of hydrocodone bitartrate and acetaminophen tablets, especially by children, can result in a fatal overdose of hydrocodone bitartrate and acetaminophen tablets [see WARNINGS ]. Neonatal Opioid Withdrawal Syndrome Prolonged use of hydrocodone bitartrate and acetaminophen tablets during pregnancy can result in neonatal opioid withdrawal syndrome, which may be life-threatening if not recognized and treated, and requires management according to protocols developed by neonatology experts. If opioid use is required for a prolonged period in a pregnant woman, advise the patient of the risk of neonatal opioid withdrawal syndrome and ensure that appropriate treatment will be available [see WARNINGS ]. Cytochrome P450 3A4 Interaction The concomitant use of hydrocodone bitartrate and acetaminophen tablets with all cytochrome P450 3A4 inhibitors may result in an increase in hydrocodone plasma concentrations, which could increase or prolong adverse reactions and may cause potentially fatal respiratory depression. In addition, discontinuation of a concomitantly used cytochrome P450 3A4 inducer may result in an increase in hydrocodone plasma concentrations. Monitor patients receiving hydrocodone bitartrate and acetaminophen tablets and any cytochrome P450 3A4 inhibitor or inducer for signs of respiratory depression or sedation [see CLINICAL PHARMACOLOGY , WARNINGS , PRECAUTIONS ; Drug Interactions ]. Hepatotoxicity Acetaminophen has been associated with cases of acute liver failure, at times resulting in liver transplant and death. Most of the cases of liver injury are associated with the use of acetaminophen at doses that exceed 4,000 milligrams per day, and often involve more than one acetaminophen containing product [see WARNINGS , OVERDOSAGE ]. Risks from Concomitant Use with Benzodiazepines or Other CNS Depressants Concomitant us e of opioids with benzodiazepines or other central nervous system (CNS) depressants , including alcohol, may result in profound sedation, respiratory depression, coma, and death [see WARNINGS , PRECAUTIONS ; Drug Interactions ]. \u2022 Reserve concomitant prescribing of hydrocodone bitartrate and acetaminophen tablets and benzodiazepines or other CNS depressants for use in patients for whom alternative treatment options are inadequate. \u2022 Limit dosages and durations to the minimum required. \u2022 Follow patients for signs and symptoms of respiratory depression and sedation."
      ],
      "description": [
        "DESCRIPTION Hydrocodone bitartrate and acetaminophen are available in tablet form for oral administration. Hydrocodone bitartrate is an opioid analgesic and occurs as fine, white crystals or as a crystalline powder. It is affected by light. The chemical name is 4,5\u03b1-epoxy-3-methoxy-17-methylmorphinan-6-one tartrate (1:1) hydrate (2:5). It has the following structural formula: Acetaminophen, 4' -hydroxyacetanilide, a slightly bitter, white, odorless, crystalline powder, is a nonopiate, non-salicylate analgesic and antipyretic. It has the following structural formula: Hydrocodone Bitartate and Acetaminophen Tablets USP is available in the following strengths: Acetaminophen USP\u2026\u2026\u2026\u2026\u2026\u2026\u2026. 300 mg Hydrocodone Bitartrate USP\u2026\u2026\u2026\u2026 5 mg WARNING: May be habit-forming. Acetaminophen USP\u2026\u2026\u2026\u2026\u2026\u2026\u2026. 300 mg Hydrocodone Bitartrate USP\u2026\u2026\u2026\u2026 7.5 mg WARNING: May be habit-forming. Acetaminophen USP\u2026\u2026\u2026\u2026\u2026\u2026\u2026. 300 mg Hydrocodone Bitartrate USP\u2026\u2026\u2026\u2026 10 mg WARNING: May be habit-forming. In addition, each tablet contains the following inactive ingredients: colloidal silicon dioxide, croscarmellose sodium, crospovidone, magnesium stearate, microcrystalline cellulose, povidone, pregelatinized starch (maize) and stearic acid. This product complies with USP Dissolution Test 1. Chemical Structure Chemical Structure"
      ],
      "clinical_pharmacology": [
        "CLINICAL PHARMACOLOGY Mechanism of Action Hydrocodone is a full opioid agonist with relative selectivity for the mu-opioid (\u03bc) receptor, although it clifean interact with other opioid receptors at higher doses. The principal therapeutic action of hydrocodone is analgesia. Like all full opioid agonists, there is no ceiling effect for analgesia with hydrocodone. Clinically, dosage is titrated to provide adequate analgesia and may be limited by adverse reactions, including respiratory and CNS depression. The precise mechanism of the analgesic action is unknown. However, specific CNS opioid receptors for endogenous compounds with opioid-like activity have been identified throughout the brain and spinal cord and are thought to play a role in the analgesic effects of this drug. The precise mechanism of the analgesic properties of acetaminophen is not established but is thought to involve central actions. Pharmacodynamics Effects on the Central Nervous System The principal therapeutic action of hydrocodone is analgesia. Hydrocodone produces respiratory depression by direct action on brain stem respiratory centers. The respiratory depression involves a reduction in the responsiveness of the brain stem respiratory centers to both increases in carbon dioxide tension and electrical stimulation. Hydrocodone causes miosis, even in total darkness. Pinpoint pupils are a sign of opioid overdose but are not pathognomonic (e.g., pontine lesions of hemorrhagic or ischemic origins may produce similar findings). Marked mydriasis rather than miosis may be seen due to hypoxia in overdose situations. Therapeutic doses of acetaminophen have negligible effects on the cardiovascular or respiratory systems; however, toxic doses may cause circulatory failure and rapid, shallow breathing. Effects on the Gastrointestinal Tract and Other Smooth Muscle Hydrocodone causes a reduction in motility associated with an increase in smooth muscle tone in the antrum of the stomach and duodenum. Digestion of food in the small intestine is delayed and propulsive contractions are decreased. Propulsive peristaltic waves in the colon are decreased, while tone may be increased to the point of spasm, resulting in constipation. Other opioid-induced effects may include a reduction in biliary and pancreatic secretions, spasm of sphincter of Oddi, and transient elevations in serum amylase. Effects on the Cardiovascular System Hydrocodone produces peripheral vasodilation which may result in orthostatic hypotension or syncope. Manifestations of histamine release and/or peripheral vasodilation may include pruritus, flushing, red eyes, sweating, and/or orthostatic hypotension. Effects on the Endocrine System Opioids inhibit the secretion of adrenocorticotropic hormone (ACTH), cortisol, and luteinizing hormone (LH) in humans [see ADVERSE REACTIONS ]. They also stimulate prolactin, growth hormone (GH) secretion, and pancreatic secretion of insulin and glucagon. Chronic use of opioids may influence the hypothalamic-pituitary-gonadal axis, leading to androgen deficiency that may manifest as symptoms as low libido, impotence, erectile dysfunction, amenorrhea, or infertility. The causal role of opioids in the syndrome of hypogonadism is unknown because the various medical, physical, lifestyle, and psychological stressors that may influence gonadal hormone levels have not been adequately controlled for in studies conducted to date [see ADVERSE REACTIONS ] . Effects on the Immune System Opioids have been shown to have a variety of effects on components of the immune system. The clinical significance of these findings is unknown. Overall, the effects of opioids appear to be modestly immunosuppressive. Concentration-Efficacy Relationships The minimum effective analgesic concentration will vary widely among patients, especially among patients who have been previously treated with potent agonist opioids. The minimum effective analgesic concentration of hydrocodone for any individual patient may increase over time due to an increase in pain, the development of a new pain syndrome, and/or the development of analgesic tolerance [see DOSAGE AND ADMINISTRATION ]. Concentration-Adverse Reaction Relationships There is a relationship between increasing hydrocodone plasma concentration and increasing frequency of dose-related opioid adverse reactions such as nausea, vomiting, CNS effects, and respiratory depression. In opioid-tolerant patients, the situation may be altered by the development of tolerance to opioid-related adverse reactions [see DOSAGE AND ADMINISTRATION ]. Pharmacokinetics The behavior of the individual components is described below. Hydrocodone Following a 10 mg oral dose of hydrocodone administered to five adult male subjects, the mean peak concentration was 23.6 \u00b1 5.2 ng/mL. Maximum serum levels were achieved at 1.3 \u00b1 0.3 hours and the half-life was determined to be 3.8 \u00b1 0.3 hours. Hydrocodone exhibits a complex pattern of metabolism including O-demethylation, N-demethylation and 6-keto reduction to the corresponding 6-\u03b1- and 6-\u03b2-hydroxymetabolites. See OVERDOSAGE for toxicity information. CYP3A4 mediated N-demethylation to norhydrocodone is the primary metabolic pathway of hydrocodone with a lower contribution from CYP2D6 mediated O-demethylation to hydromorphone. Hydromorphone is formed from the O-demethylation of hydrocodone and may contribute to the total analgesic effect of hydrocodone. Therefore, the formation of these and related metabolites can, in theory, be affected by other drugs [see PRECAUTIONS ; Drug Interactions ]. N-demethylation of hydrocodone to form norhydrocodone via CYP3A4 while O-demethylation of hydrocodone to hydromorphone is predominantly catalyzed by CYP2D6 and to a lesser extent by an unknown low affinity CYP enzyme. Hydrocodone and its metabolites are eliminated primarily in the kidneys. Acetaminophen Acetaminophen is rapidly absorbed from the gastrointestinal tract and is distributed throughout most body tissues. A small fraction (10-25%) of acetaminophen is bound to plasma proteins. The plasma halflife is 1.25 to 3 hours, but may be increased by liver damage and following overdosage. Elimination of acetaminophen is principally by liver metabolism (conjugation) and subsequent renal excretion of metabolites. Acetaminophen is primarily metabolized in the liver by first-order kinetics and involves three principal separate pathways: conjugation with glucuronide; conjugation with sulfate; and oxidation via the cytochrome, P450-dependent, mixed-function oxidase enzyme pathway to form a reactive intermediate metabolite, which conjugates with glutathione and is then further metabolized to form cysteine and mercapturic acid conjugates. The principal cytochrome P450 isoenzyme involved appears to be CYP2E1, with CYP1A2 and CYP3A4 as additional pathways. Approximately 85% of an oral dose appears in the urine within 24 hours of administration, most as the glucuronide conjugate, with small amounts of other conjugates and unchanged drug. See OVERDOSAGE for toxicity information."
      ],
      "indications_and_usage": [
        "INDICATIONS AND USAGE Hydrocodone bitartrate and acetaminophen tablets are indicated for the management of pain severe enough to require an opioid analgesic and for which alternative treatments are inadequate. Limitations of Use Because of the risks of addiction, abuse, and misuse, with opioids, even at recommended doses [see WARNINGS ], reserve hydrocodone bitartrate and acetaminophen tablets for use in patients for whom alternative treatment options (e.g., non-opioid analgesics): have not been tolerated, or are not expected to be tolerated, have not provided adequate analgesia, or are not expected to provide adequate analgesia."
      ],
      "contraindications": [
        "CONTRAINDICATIONS Hydrocodone Bitartrate and acetaminophen tablets are contraindicated in patients with: Significant respiratory depression [see WARNINGS ] Acute or severe bronchial asthma in an unmonitored setting or in the absence of resuscitative equipment [see WARNINGS ] Known or suspected gastrointestinal obstruction, including paralytic ileus [see WARNINGS ] Hypersensitivity to hydrocodone or acetaminophen (e.g., anaphylaxis) [see WARNINGS , ADVERSE REACTIONS ]"
      ],
      "warnings": [
        "WARNINGS Addiction, Abuse, and Misuse Hydrocodone bitartrate and acetaminophen tablets contain hydrocodone, a Schedule II controlled substance. As an opioid, hydrocodone bitartrate and acetaminophen tablets expose users to the risks of addiction, abuse, and misuse [see DRUG ABUSE AND DEPENDENCE ]. Although the risk of addiction in any individual is unknown, it can occur in patients appropriately prescribed hydrocodone bitartrate and acetaminophen tablets. Addiction can occur at recommended dosages and if the drug is misused or abused. Assess each patient\u2019s risk for opioid addiction, abuse, or misuse prior to prescribing hydrocodone bitartrate and acetaminophen tablets, and monitor all patients receiving hydrocodone bitartrate and acetaminophen tablets for the development of these behaviors and conditions. Risks are increased in patients with a personal or family history of substance abuse (including drug or alcohol abuse or addiction) or mental illness (e.g., major depression). The potential for these risks should not, however, prevent the proper management of pain in any given patient. Patients at increased risk may be prescribed opioids such as hydrocodone bitartrate and acetaminophen tablets, but use in such patients necessitates intensive counseling about the risks and proper use of hydrocodone bitartrate and acetaminophen tablets along with intensive monitoring for signs of addiction, abuse, and misuse. Consider prescribing naloxone for the emergency treatment of opioid overdose [see WARNINGS, Life-Threatening Respiratory Depression ; Dosage and Administration, Patient Access to Naloxone for the Emergency Treatment of Opioid Overdose ]. Opioids are sought by drug abusers and people with addiction disorders and are subject to criminal diversion. Consider these risks when prescribing or dispensing hydrocodone bitartrate and acetaminophen tablets. Strategies to reduce these risks include prescribing the drug in the smallest appropriate quantity and advising the patient on the proper disposal of unused drug [see PRECAUTIONS ; Information for Patients ]. Contact local state professional licensing board or state controlled substances authority for information on how to prevent and detect abuse or diversion of this product. Opioid Analgesic Risk Evaluation and Mitigation Strategy (REMS) To ensure that the benefits of opioid analgesics outweigh the risks of addiction, abuse, and misuse, the Food and Drug Administration (FDA) has required a Risk Evaluation and Mitigation Strategy (REMS) for these products. Under the requirements of the REMS, drug companies with approved opioid analgesic products must make REMS-compliant education programs available to healthcare providers. Healthcare providers are strongly encouraged to do all of the following: \u2022 Complete a REMS-compliant education program offered by an accredited provider of continuing education (CE) or another education program that includes all the elements of the FDA Education Blueprint for Health Care Providers Involved in the Management or Support of Patients with Pain. \u2022 Discuss the safe use, serious risks, and proper storage and disposal of opioid analgesics with patients and/or their caregivers every time these medicines are prescribed. The Patient Counseling Guide (PCG) can be obtained at this link: www.fda.gov/OpioidAnalgesicREMSPCG. \u2022 Emphasize to patients and their caregivers the importance of reading the Medication Guide that they will receive from their pharmacist every time an opioid analgesic is dispensed to them. \u2022 Consider using other tools to improve patient, household, and community safety, such as patient-prescriber agreements that reinforce patient-prescriber responsibilities. To obtain further information on the opioid analgesic REMS and for a list of accredited REMS CME/CE, call 800-503-0784, or log on to www.opioidanalgesicrems.com. The FDA Blueprint can be found at www.fda.gov/OpioidAnalgesicREMSBlueprint. Life-Threatening Respiratory Depression Serious, life-threatening, or fatal respiratory depression has been reported with the use of opioids, even when used as recommended. Respiratory depression, if not immediately recognized and treated, may lead to respiratory arrest and death. Management of respiratory depression may include close observation, supportive measures, and use of opioid antagonists, depending on the patient\u2019s clinical status [see OVERDOSAGE ]. Carbon dioxide (CO) retention from opioid-induced respiratory depression can exacerbate the sedating effects of opioids. While serious, life-threatening, or fatal respiratory depression can occur at any time during the use of hydrocodone bitartrate and acetaminophen tablets, the risk is greatest during the initiation of therapy or following a dosage increase. Monitor patients closely for respiratory depression, especially within the first 24-72 hours of initiating therapy with and following dosage increases of hydrocodone bitartrate and acetaminophen tablets. To reduce the risk of respiratory depression, proper dosing and titration of hydrocodone bitartrate and acetaminophen tablets are essential [see DOSAGE AND ADMINISTRATION ]. Overestimating the hydrocodone bitartrate and acetaminophen tablets dosage when converting patients from another opioid product can result in a fatal overdose. Accidental ingestion of even one dose of hydrocodone bitartrate and acetaminophen tablets, especially by children, can result in respiratory depression and death due to an overdose of hydrocodone and acetaminophen. Educate patients and caregivers on how to recognize respiratory depression and emphasize the importance of calling 911 or getting emergency medical help right away in the event of a known or suspected overdose [see PRECAUTIONS, Information for Patients ]. Opioids can cause sleep-related breathing disorders including central sleep apnea (CSA) and sleep-related hypoxemia. Opioid use increases the risk of CSA in a dose-dependent fashion. In patients who present with CSA, consider decreasing the opioid dosage using best practices for opioid taper [see Dosage and Administration ]. Patient Access to Naloxone for the Emergency Treatment of Opioid Overdose Discuss the availability of naloxone for the emergency treatment of opioid overdose with the patient and caregiver and assess the potential need for access to naloxone, both when initiating and renewing treatment with hydrocodone bitartrate and acetaminophen tablets. Inform patients and caregivers about the various ways to obtain naloxone as permitted by individual state naloxone dispensing and prescribing requirements or guidelines (e.g., by prescription, directly from a pharmacist, or as part of a community-based program). Educate patients and caregivers on how to recognize respiratory depression and emphasize the importance of calling 911 or getting emergency medical help, even if naloxone is administered [see PRECAUTIONS, Information for Patients ]. Consider prescribing naloxone, based on the patient\u2019s risk factors for overdose, such as concomitant use of other CNS depressants, a history of opioid use disorder, or prior opioid overdose. The presence of risk factors for overdose should not prevent the proper management of pain in any given patient. Also consider prescribing naloxone if the patient has household members (including children) or other close contacts at risk for accidental ingestion or overdose. If naloxone is prescribed, educate patients and caregivers on how to treat with naloxone [see WARNINGS, Addiction, Abuse, and Misuse, Risks from Concomitant Use with Benzodiazepines or Other CNS Depressants ; PRECAUTIONS, Information for Patients ]. Neonatal Opioid Withdrawal Syndrome Prolonged use of hydrocodone bitartrate and acetaminophen tablets during pregnancy can result in withdrawal in the neonate. Neonatal opioid withdrawal syndrome, unlike opioid withdrawal syndrome in adults, may be life-threatening if not recognized and treated, and requires management according to protocols developed by neonatology experts. Observe newborns for signs of neonatal opioid withdrawal syndrome and manage accordingly. Advise pregnant women using opioids for a prolonged period of the risk of neonatal opioid withdrawal syndrome and ensure that appropriate treatment will be available [see PRECAUTIONS ; Information for Patients , Pregnancy ]. Risks of Concomitant Use or Discontinuation of Cytochrome P450 3A4 Inhibitors and Inducers Concomitant use of hydrocodone bitartrate and acetaminophen tablets with a CYP3A4 inhibitor, such as macrolide antibiotics (e.g., erythromycin), azole-antifungal agents (e.g., ketoconazole), and protease inhibitors (e.g., ritonavir), may increase plasma concentrations of hydrocodone bitartrate and acetaminophen tablets and prolong opioid adverse reactions, and which may cause potentially fatal respiratory depression [see WARNINGS ], particularly when an inhibitor is added after a stable dose of hydrocodone bitartrate and acetaminophen tablets is achieved. Similarly, discontinuation of a CYP3A4 inducer, such as rifampin, carbamazepine, and phenytoin, in hydrocodone bitartrate and acetaminophen tablets-treated patients may increase hydrocodone plasma concentrations and prolong opoid adverse reactions. When adding CYP3A4 inhibitors or discontinuing CYP3A4 inducers in hydrocodone bitartrate and acetaminophen tablets-treated patients, follow patients at frequent intervals and consider dosage reduction of hydrocodone bitartrate and acetaminophen tablets until stable drug effects are achieved [see PRECAUTIONS ; Drug Interactions ]. Concomitant use of hydrocodone bitartrate and acetaminophen tablets with CYP3A4 inducers or discontinuation of an CYP3A4 inhibitor could decrease hydrocodone plasma concentrations, decrease opioid efficacy or, possibly, lead to a withdrawal syndrome in a patient who had developed physical dependence to hydrocodone. When using bydrocodone bitartrate and acetaminophen tablets with CYP3A4 inducers or discontinuing CYP3A4 inhibitors, follow patients at frequent intervals and consider increasing the opioid dosage if needed to maintain adequate analgesia or if symptoms of opioid withdrawal occur [see PRECAUTIONS ; Drug Interactions ]. Risks from Concomitant Use with Benzodiazepines or Other CNS Depressants Profound sedation, respiratory depression, coma, and death may result from the concomitant use of hydrocodone bitartrate and acetaminophen tablets with benzodiazepines or other CNS depressants (e.g., non-benzodiazepine sedatives/hypnotics, anxiolytics, tranquilizers, muscle relaxants, general anesthetics, antipsychotics, other opioids, alcohol). Because of these risks, reserve concomitant prescribing of these drugs for use in patients for whom alternative treatment options are inadequate. Observational studies have demonstrated that concomitant use of opioid analgesics and benzodiazepines increases the risk of drug-related mortality compared to use of opioid analgesics alone. Because of similar pharmacological properties, it is reasonable to expect similar risk with the concomitant use of other CNS depressant drugs with opioid analgesics [see PRECAUTIONS ; Drug Interactions ]. If the decision is made to prescribe a benzodiazepine or other CNS depressant concomitantly with an opioid analgesic, prescribe the lowest effective dosages and minimum durations of concomitant use. In patients already receiving an opioid analgesic, prescribe a lower initial dose of the benzodiazepine or other CNS depressant than indicated in the absence of an opioid, and titrate based on clinical response. If an opioid analgesic is initiated in a patient already taking a benzodiazepine or other CNS depressant, prescribe a lower initial dose of the opioid analgesic, and titrate based on clinical response. Follow patients closely for signs and symptoms of respiratory depression and sedation. If concomitant use is warranted, consider prescribing naloxone for the emergency treatment of opioid overdose [see WARNINGS, Life-Threatening Respiratory Depression; Dosage and Administration, Patient Access to Naloxone for the Emergency Treatment of Opioid Overdose]. Advise both patients and caregivers about the risks of respiratory depression and sedation when hydrocodone bitartrate and acetaminophen tablets are used with benzodiazepines or other CNS depressants (including alcohol and illicit drugs). Advise patients not to drive or operate heavy machinery until the effects of concomitant use of the benzodiazepine or other CNS depressant have been determined. Screen patients for risk of substance use disorders, including opioid abuse and misuse, and warn them of the risk for overdose and death associated with the use of additional CNS depressants including alcohol and illicit drugs [see PRECAUTIONS ; Drug Interactions , Information for Patients ]. Life-Threatening Respiratory Depression in Patients with Chronic Pulmonary Disease or in Elderly, Cachectic, or Debilitated Patients The use of hydrocodone bitartrate and acetaminophen tablets in patients with acute or severe bronchial asthma in an unmonitored setting or in the absence of resuscitative equipment is contraindicated. Patients with Chronic Pulmonary Disease: Hydrocodone bitartrate and acetaminophen tablet-treated patients with significant chronic obstructive pulmonary disease or cor pulmonale, and those with a substantially decreased respiratory reserve, hypoxia, hypercapnia, or pre-existing respiratory depression are at increased risk of decreased respiratory drive including apnea, even at recommended dosages of hydrocodone bitartrate and acetaminophen tablets [see WARNINGS ; Life-Threatening Respiratory Depression ]. Elderly, Cachetic, or Debilitated Patients: Life-threatening respiratory depression is more likely to occur in elderly, cachectic, or debilitated patients because they may have altered pharmacokinetics or altered clearance compared to younger, healthier patients [see WARNINGS ; Life-Threatening Respiratory Depression ]. Follow such patients closely, particularly when initiating and titrating hydrocodone bitartrate and acetaminophen tablets and when hydrocodone bitartrate and acetaminophen tablets are given concomitantly with other drugs that depress respiration [see WARNINGS ; Life-Threatening Respiratory Depression ]. Alternatively, consider the use of non-opioid analgesics in these patients. Adrenal Insufficiency Cases of adrenal insufficiency have been reported with opioid use, more often following greater than one month of use. Presentation of adrenal insufficiency may include non-specific symptoms and signs including nausea, vomiting, anorexia, fatigue, weakness, dizziness, and low blood pressure. If adrenal insufficiency is suspected, confirm the diagnosis with diagnostic testing as soon as possible. If adrenal insufficiency is diagnosed, treat with physiologic replacement doses of corticosteroids. Wean the patient off of the opioid to allow adrenal function to recover and continue corticosteroid treatment until adrenal function recovers. Other opioids may be tried as some cases reported use of a different opioid without recurrence of adrenal insufficiency. The information available does not identify any particular opioids as being more likely to be associated with adrenal insufficiency. Severe Hypotension Hydrocodone bitartrate and acetaminophen tablets may cause severe hypotension including orthostatic hypotension and syncope in ambulatory patients. There is increased risk in patients whose ability to maintain blood pressure has already been compromised by a reduced blood volume or concurrent administration of certain CNS depressant drugs (e.g., phenothiazines or general anesthetics) [see PRECAUTIONS ; Drug Interactions ]. Follow these patients for signs of hypotension after initiating or titrating the dosage of hydrocodone bitartrate and acetaminophen tablets. In patients with circulatory shock hydrocodone bitartrate and acetaminophen tablets may cause vasodilatation that can further reduce cardiac output and blood pressure. Avoid the use of hydrocodone bitartrate and acetaminophen tablets with circulatory shock. Hepatotoxicity Acetaminophen has been associated with cases of acute liver failure, at times resulting in liver transplant and death. Most of the cases of liver injury are associated with the use of acetaminophen at doses that exceed 4,000 milligrams per day, and often involve more than one acetaminophen-containing product. The excessive intake of acetaminophen may be intentional to cause self-harm or unintentional as patients attempt to obtain more pain relief or unknowingly take other acetaminophen-containing products. The risk of acute liver failure is higher in individuals with underlying liver disease and in individuals who ingest alcohol while taking acetaminophen. Instruct patients to look for acetaminophen or APAP on package labels and not to use more than one product that contains acetaminophen. Instruct patients to seek medical attention immediately upon ingestion of more than 4,000 milligrams of acetaminophen per day, even if they feel well. Serious Skin Reactions Rarely, acetaminophen may cause serious skin reactions such as acute generalized exanthematous pustulosis (AGEP), Stevens-Johnson Syndrome (SJS), and toxic epidermal necrolysis (TEN), which can be fatal. Patients should be informed about the signs of serious skin reactions, and use of the drug should be discontinued at the first appearance of skin rash or any other sign of hypersensitivity. Hypersensitivity/Anaphylaxis There have been post-marketing reports of hypersensitivity and anaphylaxis associated with the use of acetaminophen. Clinical signs included swelling of the face, mouth, and throat, respiratory distress, urticaria, rash, pruritus, and vomiting. There were infrequent reports of life-threatening anaphylaxis requiring emergency medical attention. Instruct patients to discontinue hydrocodone bitartrate and acetaminophen tablets immediately and seek medical care if they experience these symptoms. Do not prescribe hydrocodone bitartrate and acetaminophen tablets for patients with acetaminophen allergy [see PRECAUTIONS ; Information for Patients/Caregivers ]. Risks of Use in Patients with Increased Intracranial Pressure, Brain Tumors, Head Injury, or Impaired Consciousness In patients who may be susceptible to the intracranial effects of CO retention (e.g., those with evidence of increased intracranial pressure or brain tumors), hydrocodone bitartrate and acetaminophen tablets may reduce respiratory drive, and the resultant CO retention can further increase intracranial pressure. Follow such patients for signs of sedation and respiratory depression, particularly when initiating therapy with hydrocodone bitartrate and acetaminophen tablets. Opioids may also obscure the clinical course in a patient with a head injury. Avoid the use of hydrocodone bitartrate and acetaminophen tablets in patients with impaired consciousness or coma. Risks of Use in Patients with Gastrointestinal Conditions Hydrocodone bitartrate and acetaminophen tablets are contraindicated in patients with gastrointestinal obstruction, including paralytic ileus. The administration of hydrocodone bitartrate and acetaminophen tablets or other opioids may obscure the diagnosis or clinical course in patients with acute abdominal conditions. Hydrocodone may cause spasm of the sphincter of Oddi. Opioids may cause increases in serum amylase. Monitor patients with biliary tract disease, including acute pancreatitis, for worsening symptoms. Increased Risk of Seizures in Patients with Seizure Disorders The hydrocodone in hydrocodone bitartrate and acetaminophen tablets may increase the frequency of seizures in patients with seizure disorders, and may increase the risk of seizures occurring in other clinical settings associated with seizures. Follow patients with a history of seizure disorders for worsened seizure control during hydrocodone bitartrate and acetaminophen tablet therapy. Withdrawal Do not abruptly discontinue Hydrocodone Bitartrate and AcetaminophenTablets in a patient physically dependent on opioids. When discontinuing Hydrocodone Bitartrate and Acetaminophen Tablets in a physically dependent patient, gradually taper the dosage. Rapid tapering of Hydrocodone Bitartrate and AcetaminophenTablets in a patient physically dependent on opioids may lead to a withdrawal syndrome and return of pain [see Dosage and Administration , Drug Abuse and Dependence ]. Additionally, avoid the use of mixed agonist/antagonist (e.g., pentazocine, nalbuphine, and butorphanol) or partial agonist (e.g., buprenorphine) analgesics in patients who are receiving a full opioid agonist analgesic, including Hydrocodone Bitartrate and AcetaminophenTablets. In these patients, mixed agonist/antagonist and partial agonist analgesics may reduce the analgesic effect and/or precipitate withdrawal symptoms [see PRECAUTIONS / Drug Interactions ]."
      ],
      "precautions": [
        "PRECAUTIONS Risks of Driving and Operating Machinery Hydrocodone bitartrate and acetaminophen tablets may impair the mental or physical abilities needed to perform potentially hazardous activities such as driving a car or operating machinery. Warn patients not to drive or operate dangerous machinery unless they are tolerant to the effects of hydrocodone bitartrate and acetaminophen tablets and know how they will react to the medication [see PRECAUTIONS ; Information for Patients /Caregivers ]. Information for Patients Advise the patient to read the FDA-approved patient labeling (Medication Guide). Storage and Disposal Because of the risks associated with accidental ingestion, misuse, and abuse, advise patients to store Hydrocodone Bitartrate and Acetaminophen Tablets securely, out of sight and reach of children, and in a location not accessible by others, including visitors to the home [see WARNINGS , DRUG ABUSE AND DEPENDENCE ]. Inform patients that leaving Hydrocodone Bitartrate and Acetaminophen Tablets unsecured can pose a deadly risk to others in the home. Advise patients and caregivers that when medicines are no longer needed, they should be disposed of promptly. Expired, unwanted, or unused Hydrocodone Bitartrate and Acetaminophen Tablets should be disposed of by flushing the unused medication down the toilet if a drug take-back option is not readily available. Inform patients that they can visit www.fda.gov/drugdisposal for a complete list of medicines recommended for disposal by flushing, as well as additional information on disposal of unused medicines. Addiction, Abuse, and Misuse Inform patients that the use of hydrocodone bitartrate and acetaminophen tablets, even when taken as recommended, can result in addiction, abuse, and misuse, which can lead to overdose and death [see WARNINGS ]. Instruct patients not to share hydrocodone bitartrate and acetaminophen tablets with others and to take steps to protect hydrocodone bitartrate and acetaminophen tablets from theft or misuse. Life-Threatening Respiratory Depression Inform patients of the risk of life-threatening respiratory depression, including information that the risk is greatest when starting hydrocodone bitartrate and acetaminophen tablets or when the dosage is increased, and that it can occur even at recommended dosages.Advise patients how to recognize respiratory depression and to seek medical attention if breathing difficulties develop. Educate patients and caregivers on how to recognize respiratory depression and emphasize the importance of calling 911 or getting emergency medical help right away in the event of a known or suspected overdose [see WARNINGS, Life Threatening Respiratory Depression ]. Patient Access to Naloxone for the Emergency Treatment of Opioid Overdose Discuss with the patient and caregiver the availability of naloxone for the emergency treatment of opioid overdose, both when initiating and renewing treatment with hydrocodone bitartrate and acetaminophen tablets. Inform patients and caregivers about the various ways to obtain naloxone as permitted by individual state naloxone dispensing and prescribing requirements or guidelines (e.g., by prescription, directly from a pharmacist, or as part of a community-based program) [see WARNINGS, Life-Threatening Respiratory Depression ; DOSAGE AND ADMINISTRATION ]. Educate patients and caregivers on how to recognize the signs and symptoms of an overdose. Explain to patients and caregivers that naloxone\u2019s effects are temporary, and that they must call 911 or get emergency medical help right away in all cases of known or suspected opioid overdose, even if naloxone is administered [see OVERDOSAGE ]. If naloxone is prescribed, also advise patients and caregivers: \u2022 How to treat with naloxone in the event of an opioid overdose \u2022 To tell family and friends about their naloxone and to keep it in a place where family and friends can access it in an emergency \u2022 To read the Patient Information (or other educational material) that will come with their naloxone. Emphasize the importance of doing this before an opioid emergency happens, so the patient and caregiver will know what to do. Accidental Ingestion Inform patients that accidental ingestion, especially by children, may result in respiratory depression or death [see WARNINGS ]. Interactions with Benzodiazepines and Other CNS Depressants Inform patients and caregivers that potentially fatal additive effects may occur if hydrocodone bitartrate and acetaminophen tablets are used with benzodiazepines and other CNS depressants, including alcohol, and not to use these concomitantly unless supervised by a healthcare provider [see WARNINGS , PRECAUTIONS ; Drug Interactions ]. Serotonin Syndrome Inform patients that hydrocodone bitartrate and acetaminophen tablets could cause a rare but potentially life-threatening condition resulting from concomitant administration of serotonergic drugs. Warn patients of the symptoms of serotonin syndrome and to seek medical attention right away if symptoms develop. Instruct patients to inform their healthcare providers if they are taking, or plan to take serotonergic medications [see PRECAUTIONS ; Drug Interactions ]. Monoamine Oxidase Inhibitor (MAOI) Interaction Inform patients to avoid taking hydrocodone bitartrate and acetaminophen tablets while using any drugs that inhibit monoamine oxidase. Patients should not start MAOIs while taking hydrocodone bitartrate and acetaminophen tablets [see PRECAUTIONS ; Drug Interactions ]. Adrenal Insufficiency Inform patients that hydrocodone bitartrate and acetaminophen tablets opioids could cause adrenal insufficiency, a potentially life-threatening condition. Adrenal insufficiency may present with non-specific symptoms and signs such as nausea, vomiting, anorexia, fatigue, weakness, dizziness, and low blood pressure. Advise patients to seek medical attention if they experience a constellation of these symptoms [see WARNINGS ]. Important Administration Instructions Instruct patients how to properly take hydrocodone bitartrate and acetaminophen tablets [see DOSAGE AND ADMINISTRATION , WARNINGS ]. Important Discontinuation Instructions In order to avoid developing withdrawal symptoms, instruct patients not to discontinue Hydrocodone Bitartrate and Acetaminophen Tablets without first discussing a tapering plan with the prescriber [see DOSAGE AND ADMINISTRATION ] Maximum Daily Dose of Acetaminophen Inform patients not to take more than 4000 milligrams of acetaminophen per day. Advise patients to call their prescriber if they take more than the recommended dose. Hypotension Inform patients that hydrocodone bitartrate and acetaminophen tablets may cause orthostatic hypotension and syncope. Instruct patients how to recognize symptoms of low blood pressure and how to reduce the risk of serious consequences should hypotension occur (e.g., sit or lie down, carefully rise from a sitting or lying position) [see WARNINGS ]. Anaphylaxis Inform patients that anaphylaxis has been reported with ingredients contained in hydrocodone bitartrate and acetaminophen tablets. Advise patients how to recognize such a reaction and when to seek medical attention [see CONTRAINDICATIONS , ADVERSE REACTIONS ]. Pregnancy Neonatal Opioid Withdrawal Syndrome Inform female patients of reproductive potential that prolonged use of hydrocodone bitartrate and acetaminophen tablets during pregnancy can result in neonatal opioid withdrawal syndrome, which may be life-threatening if not recognized and treated [see WARNINGS , PRECAUTIONS ; Pregnancy ]. Embryo-Fetal Toxicity Inform female patients of reproductive potential that hydrocodone bitartrate and acetaminophen tablets can cause fetal harm and to inform their healthcare provider of a known or suspected pregnancy [see PRECAUTIONS ; Pregnancy ]. Lactation Advise nursing mothers to monitor infants for increased sleepiness (more than usual), breathing difficulties, or limpness. Instruct nursing mothers to seek immediate medical care if they notice these signs [see PRECAUTIONS ; Nursing Mothers ]. Infertility Inform patients that chronic use of opioids may cause reduced fertility. It is not known whether these effects on fertility are reversible [see ADVERSE REACTIONS ]. Driving or Operating Heavy Machinery Inform patients that hydrocodone bitartrate and acetaminophen tablets may impair the ability to perform potentially hazardous activities such as driving a car or operating heavy machinery. Advise patients not to perform such tasks until they know how they will react to the medication [see WARNINGS ]. Constipation Advise patients of the potential for severe constipation, including management instructions and when to seek medical attention [see ADVERSE REACTIONS , CLINICAL PHARMACOLOGY ]. Laboratory Tests In patients with severe hepatic or renal disease, effects of therapy should be followed with serial liver and/or renal function tests. Drug Interactions Inhibitors of CYP3A4 and CYP2D6 The concomitant use of hydrocodone bitartrate and acetaminophen tablets and CYP3A4 inhibitors, such as macrolide antibiotics (e.g., erythromycin), azole-antifungal agents (e.g. ketoconazole), and protease inhibitors (e.g., ritonavir), can increase the plasma concentration of the hydrocodone from hydrocodone bitartrate and acetaminophen tablets, resulting in increased or prolonged opioid effects. These effects could be more pronounced with concomitant use of hydrocodone bitartrate and acetaminophen tablets and both CYP3A4 and CYP2D6 inhibitors, particularly when an inhibitor is added after a stable dose of hydrocodone bitartrate and acetaminophen tablets is achieved [see WARNINGS ]. After stopping a CYP3A4 inhibitor, as the effects of the inhibitor decline, the hydrocodone plasma concentration will decrease [see CLINICAL PHARMACOLOGY ], resulting in decreased opioid efficacy or a withdrawal syndrome in patients who had developed physical dependence to hydrocodone bitartrate and acetaminophen tablets. If concomitant use is necessary, consider dosage reduction of hydrocodone bitartrate and acetaminophen tablets until stable drug effects are achieved. Follow patients for respiratory depression and sedation at frequent intervals. If a CYP3A4 inhibitor is discontinued, consider increasing the hydrocodone bitartrate and acetaminophen tablets dosage until stable drug effects are achieved. Follow for signs or symptoms of opioid withdrawal. Inducers of CYP3A4 The concomitant use of hydrocodone bitartrate and acetaminophen tablets and CYP3A4 inducers, such as rifampin, carbamazepine, and phenytoin, can decrease the plasma concentration of hydrocodone [see CLINICAL PHARMACOLOGY ], resulting in decreased efficacy or onset of a withdrawal syndrome in patients who have developed physical dependence to hydrocodone [see WARNINGS ]. After stopping a CYP3A4 inducer, as the effects of the inducer decline, the hydrocodone plasma concentration will increase [see CLINICAL PHARMACOLOGY ], which could increase or prolong both the therapeutic effects and adverse reactions, and may cause serious respiratory depression. If concomitant use is necessary, consider increasing the hydrocodone bitartrate and acetaminophen tablets dosage until stable drug effects are achieved. Follow the patient for signs and symptoms of opioid withdrawal. If a CYP3A4 inducer is discontinued, consider hydrocodone bitartrate and acetaminophen tablets dosage reduction and follow for signs of respiratory depression. Benzodiazepines and Other CNS Depressants Due to additive pharmacologic effect, the concomitant use of benzodiazepines and other CNS depressants, such as benzodiazepines and other sedative hypnotics, anxiolytics, and tranquilizers, muscle relaxants, general anesthetics, antipsychotics, and other opioids, including alcohol, can increase the risk of hypotension, respiratory depression, profound sedation, coma, and death. Reserve concomitant prescribing of these drugs for use in patients for whom alternative treatment options are inadequate. Limit dosages and durations to the minimum required. Follow patients closely for signs of respiratory depression and sedation. If concomitant use is warranted, consider prescribing naloxone for the emergency treatment of opioid overdose[see WARNINGS ]. Serotonergic Drugs The concomitant use of opioids with other drugs that affect the serotonergic neurotransmitter system, such as selective serotonin reuptake inhibitors (SSRIs), serotonin and norepinephrine reuptake inhibitors (SNRIs), tricyclic antidepressants (TCAs), triptans, 5-HT3 receptor antagonists, drugs that affect the serotonin neurotransmitter system (e.g., mirtazapine, trazodone, tramadol), certain muscle relaxants (i.e., cyclobenzaprine,metaxalone) and monoamine oxidase (MAO) inhibitors (those intended to treat psychiatric disorders and also others, such as linezolid and intravenous methylene blue), has resulted in serotonin syndrome [see PRECAUTIONS ; Information for Patients ]. If concomitant use is warranted, carefully follow the patient, particularly during treatment initiation and dose adjustment. Discontinue hydrocodone bitartrate and acetaminophen tablets if serotonin syndrome is suspected. Monoamine Oxidase Inhibitors (MAOIs) The concomitant use of opioids and MAOIs, such as phenelzine, tranylcypromine, or linezolid, may manifest as serotonin syndrome, or opioid toxicity (e.g., respiratory depression, coma) [see WARNINGS ]. The use of hydrocodone bitartrate and acetaminophen tablets is not recommended for patients taking MAOIs or within 14 days of stopping such treatment. If urgent use of an opioid is necessary, use test doses and frequent titration of small doses to treat pain while closely monitoring blood pressure and signs and symptoms of CNS and respiratory depression. Mixed Agonist/Antagonist and Partial Agonist Opioid Analgesics The concomitant use of opioids with other opioid analgesics, such as butorphanol, nalbuphine, pentazocine, may reduce the analgesic effect of hydrocodone bitartrate and acetaminophen tablets and/or precipitate withdrawal symptoms. Advise patient to avoid concomitant use of these drugs. Muscle Relaxants Hydrocodone bitartrate and acetaminophen tablets may enhance the neuromuscular blocking action of skeletal muscle relaxants and produce an increased degree of respiratory depression. If concomitant use is warranted, monitor patients for signs of respiratory depression that may be greater than otherwise expected and decrease the dosage of hydrocodone bitartrate and acetaminophen tablets and/or the muscle relaxant as necessary. Due to the risk of respiratory depression with concomitant use of skeletal muscle relaxants and opioids, consider prescribing naloxone for the emergency treatment of opioid overdose [see WARNINGS ]. Diuretics Opioids can reduce the efficacy of diuretics by inducing the release of antidiuretic hormone. If concomitant use is warranted, follow patients for signs of diminished diuresis and/or effects on blood pressure and increase the dosage of the diuretic as needed. Anticholinergic Drugs The concomitant use of anticholinergic drugs may increase risk of urinary retention and/or severe constipation, which may lead to paralytic ileus. If concomitant use is warranted, follow patients for signs and symptoms of urinary retention or reduced gastric motility when hydrocodone bitartrate and acetaminophen tablets are used concomitantly with anticholinergic drugs. Drug/Laboratory Test Interactions Acetaminophen may produce false-positive test results for urinary 5-hydroxyindoleacetic acid. Carcinogenesis, Mutagenesis, Impairment of Fertility Carcinogenesis Long-term studies to evaluate the carcinogenic potential of the combination of hydrocodone bitartrate and acetaminophen tablets have not been conducted. Long-term studies in mice and rats have been completed by the National Toxicology Program to evaluate the carcinogenic potential of acetaminophen. In 2-year feeding studies, F344/N rats and B6C3F1 mice were fed a diet containing acetaminophen up to 6000 ppm. Female rats demonstrated equivocal evidence of carcinogenic activity based on increased incidences of mononuclear cell leukemia at 0.8 times the maximum human daily dose (MHDD) of 4 grams/day, based on a body surface area comparison. In contrast, there was no evidence of carcinogenic activity in male rats that received up to 0.7 times or mice at up to 1.2-1.4 times the MHDD, based on a body surface area comparison. Mutagenesis In the published literature, acetaminophen has been reported to be clastogenic when administered at 1500 mg/kg/day to the rat model (3.6-times the MHDD, based on a body surface area comparison). In contrast, no clastogenicity was noted at a dose of 750 mg/kg/day (1.8-times the MHDD, based on a body surface area comparison), suggesting a threshold effect. Impairment of Fertility In studies conducted by the National Toxicology Program, fertility assessments with acetaminophen have been completed in Swiss CD-1 mice via a continuous breeding study. There were no effects on fertility parameters in mice consuming up to 1.7 times the MHDD of acetaminophen, based on a body surface area comparison. Although there was no effect on sperm motility or sperm density in the epididymis, there was a significant increase in the percentage of abnormal sperm in mice consuming 1.78 times the MHDD (based on a body surface comparison) and there was a reduction in the number of mating pairs producing a fifth litter at this dose, suggesting the potential for cumulative toxicity with chronic administration of acetaminophen near the upper limit of daily dosing. Published studies in rodents report that oral acetaminophen treatment of male animals at doses that are 1.2 times the MHDD and greater (based on a body surface comparison) result in decreased testicular weights, reduced spermatogenesis, reduced fertility, and reduced implantation sites in females given the same doses. These effects appear to increase with the duration of treatment. The clinical significance of these findings is not known. Infertility Chronic use of opioids may cause reduced fertility in females and males of reproductive potential. It is not known whether these effects on fertility are reversible [see ADVERSE REACTIONS ]. Pregnancy Teratogenic Effects There are no adequate and well-controlled studies in pregnant women. Hydrocodone bitartrate and acetaminophen tablets should be used during pregnancy only if the potential benefit justifies the potential risk to the fetus. Nonteratogenic Effects Fetal/Neonatal Adverse Reactions Prolonged use of opioid analgesics during pregnancy for medical or nonmedical purposes can result in physical dependence in the neonate and neonatal opioid withdrawal syndrome shortly after birth. Neonatal opioid withdrawal syndrome presents as irritability, hyperactivity, abnormal sleep pattern, high pitched cry, tremor, vomiting, diarrhea and failure to gain weight. The onset, duration, and severity of neonatal opioid withdrawal syndrome vary based on the specific opioid used, duration of use, timing and amount of last maternal use, and rate of elimination of the drug by the newborn. Observe newborns for symptoms of neonatal opioid withdrawal syndrome and manage accordingly [see WARNINGS ]. Labor or Delivery Opioids cross the placenta and may produce respiratory depression and psycho-physiologic effects in neonates. An opioid antagonist, such as naloxone, must be available for reversal of opioid-induced respiratory depression in the neonate. Hydrocodone bitartrate and acetaminophen tablets are not recommended for use in pregnant women during or immediately prior to labor, when other analgesic techniques are more appropriate. Opioid analgesics, including hydrocodone bitartrate and acetaminophen tablets, can prolong labor through actions which temporarily reduce the strength, duration, and frequency of uterine contractions. However, this effect is not consistent and may be offset by an increased rate of cervical dilation, which tends to shorten labor. Monitor neonates exposed to opioid analgesics during labor for signs of excess sedation and respiratory depression. Nursing Mothers Hydrocodone is present in human milk. The developmental and health benefits of breastfeeding should be considered along with the mother\u2019s clinical need for hydrocodone bitartrate and acetaminophen tablets and any potential adverse effects on the breastfed infant from hydrocodone bitartrate and acetaminophen tablets or from the underlying maternal condition. Infants exposed to hydrocodone bitartrate and acetaminophen tablets through breast milk should be monitored for excess sedation and respiratory depression. Withdrawal symptoms can occur in breastfed infants when maternal administration of an opioid analgesic is stopped, or when breast-feeding is stopped. Pediatric Use Safety and effectiveness of hydrocodone bitartrate and acetaminophen tablets in pediatric patients have not been established. Geriatric Use Elderly patients (aged 65 years or older) may have increased sensitivity to hydrocodone bitartrate and acetaminophen tablets. In general, use caution when selecting a dosage for an elderly patient, usually starting at the low end of the dosing range, reflecting the greater frequency of decreased hepatic, renal, or cardiac function, and of concomitant disease or other drug therapy. Respiratory depression is the chief risk for elderly patients treated with opioids, and has occurred after large initial doses were administered to patients who were not opioid-tolerant or when opioids were co-administered with other agents that depress respiration. Titrate the dosage of hydrocodone bitartrate and acetaminophen tablets slowly in geriatric patients and follow closely for signs of central nervous system and respiratory depression [see WARNINGS ]. Hydrocodone and acetaminophen are known to be substantially excreted by the kidney, and the risk of adverse reactions to this drug may be greater in patients with impaired renal function. Because elderly patients are more likely to have decreased renal function, care should be taken in dose selection, and it may be useful to monitor renal function. Hepatic Impairment Patients with hepatic impairment may have higher plasma hydrocodone concentrations than those with normal function. Use a low initial dose of hydrocodone bitartrate and acetaminophen tablets in patients with hepatic impairment and follow closely for adverse events such as respiratory depression and sedation. Renal Impairment Patients with renal impairment may have higher plasma hydrocodone concentrations than those with normal function. Use a low initial dose hydrocodone bitartrate and acetaminophen tablets in patients with renal impairment and follow closely for adverse events such as respiratory depression and sedation."
      ],
      "information_for_patients": [
        "Information for Patients Advise the patient to read the FDA-approved patient labeling (Medication Guide). Storage and Disposal Because of the risks associated with accidental ingestion, misuse, and abuse, advise patients to store Hydrocodone Bitartrate and Acetaminophen Tablets securely, out of sight and reach of children, and in a location not accessible by others, including visitors to the home [see WARNINGS , DRUG ABUSE AND DEPENDENCE ]. Inform patients that leaving Hydrocodone Bitartrate and Acetaminophen Tablets unsecured can pose a deadly risk to others in the home. Advise patients and caregivers that when medicines are no longer needed, they should be disposed of promptly. Expired, unwanted, or unused Hydrocodone Bitartrate and Acetaminophen Tablets should be disposed of by flushing the unused medication down the toilet if a drug take-back option is not readily available. Inform patients that they can visit www.fda.gov/drugdisposal for a complete list of medicines recommended for disposal by flushing, as well as additional information on disposal of unused medicines. Addiction, Abuse, and Misuse Inform patients that the use of hydrocodone bitartrate and acetaminophen tablets, even when taken as recommended, can result in addiction, abuse, and misuse, which can lead to overdose and death [see WARNINGS ]. Instruct patients not to share hydrocodone bitartrate and acetaminophen tablets with others and to take steps to protect hydrocodone bitartrate and acetaminophen tablets from theft or misuse. Life-Threatening Respiratory Depression Inform patients of the risk of life-threatening respiratory depression, including information that the risk is greatest when starting hydrocodone bitartrate and acetaminophen tablets or when the dosage is increased, and that it can occur even at recommended dosages.Advise patients how to recognize respiratory depression and to seek medical attention if breathing difficulties develop. Educate patients and caregivers on how to recognize respiratory depression and emphasize the importance of calling 911 or getting emergency medical help right away in the event of a known or suspected overdose [see WARNINGS, Life Threatening Respiratory Depression ]. Patient Access to Naloxone for the Emergency Treatment of Opioid Overdose Discuss with the patient and caregiver the availability of naloxone for the emergency treatment of opioid overdose, both when initiating and renewing treatment with hydrocodone bitartrate and acetaminophen tablets. Inform patients and caregivers about the various ways to obtain naloxone as permitted by individual state naloxone dispensing and prescribing requirements or guidelines (e.g., by prescription, directly from a pharmacist, or as part of a community-based program) [see WARNINGS, Life-Threatening Respiratory Depression ; DOSAGE AND ADMINISTRATION ]. Educate patients and caregivers on how to recognize the signs and symptoms of an overdose. Explain to patients and caregivers that naloxone\u2019s effects are temporary, and that they must call 911 or get emergency medical help right away in all cases of known or suspected opioid overdose, even if naloxone is administered [see OVERDOSAGE ]. If naloxone is prescribed, also advise patients and caregivers: \u2022 How to treat with naloxone in the event of an opioid overdose \u2022 To tell family and friends about their naloxone and to keep it in a place where family and friends can access it in an emergency \u2022 To read the Patient Information (or other educational material) that will come with their naloxone. Emphasize the importance of doing this before an opioid emergency happens, so the patient and caregiver will know what to do. Accidental Ingestion Inform patients that accidental ingestion, especially by children, may result in respiratory depression or death [see WARNINGS ]. Interactions with Benzodiazepines and Other CNS Depressants Inform patients and caregivers that potentially fatal additive effects may occur if hydrocodone bitartrate and acetaminophen tablets are used with benzodiazepines and other CNS depressants, including alcohol, and not to use these concomitantly unless supervised by a healthcare provider [see WARNINGS , PRECAUTIONS ; Drug Interactions ]. Serotonin Syndrome Inform patients that hydrocodone bitartrate and acetaminophen tablets could cause a rare but potentially life-threatening condition resulting from concomitant administration of serotonergic drugs. Warn patients of the symptoms of serotonin syndrome and to seek medical attention right away if symptoms develop. Instruct patients to inform their healthcare providers if they are taking, or plan to take serotonergic medications [see PRECAUTIONS ; Drug Interactions ]. Monoamine Oxidase Inhibitor (MAOI) Interaction Inform patients to avoid taking hydrocodone bitartrate and acetaminophen tablets while using any drugs that inhibit monoamine oxidase. Patients should not start MAOIs while taking hydrocodone bitartrate and acetaminophen tablets [see PRECAUTIONS ; Drug Interactions ]. Adrenal Insufficiency Inform patients that hydrocodone bitartrate and acetaminophen tablets opioids could cause adrenal insufficiency, a potentially life-threatening condition. Adrenal insufficiency may present with non-specific symptoms and signs such as nausea, vomiting, anorexia, fatigue, weakness, dizziness, and low blood pressure. Advise patients to seek medical attention if they experience a constellation of these symptoms [see WARNINGS ]. Important Administration Instructions Instruct patients how to properly take hydrocodone bitartrate and acetaminophen tablets [see DOSAGE AND ADMINISTRATION , WARNINGS ]. Important Discontinuation Instructions In order to avoid developing withdrawal symptoms, instruct patients not to discontinue Hydrocodone Bitartrate and Acetaminophen Tablets without first discussing a tapering plan with the prescriber [see DOSAGE AND ADMINISTRATION ] Maximum Daily Dose of Acetaminophen Inform patients not to take more than 4000 milligrams of acetaminophen per day. Advise patients to call their prescriber if they take more than the recommended dose. Hypotension Inform patients that hydrocodone bitartrate and acetaminophen tablets may cause orthostatic hypotension and syncope. Instruct patients how to recognize symptoms of low blood pressure and how to reduce the risk of serious consequences should hypotension occur (e.g., sit or lie down, carefully rise from a sitting or lying position) [see WARNINGS ]. Anaphylaxis Inform patients that anaphylaxis has been reported with ingredients contained in hydrocodone bitartrate and acetaminophen tablets. Advise patients how to recognize such a reaction and when to seek medical attention [see CONTRAINDICATIONS , ADVERSE REACTIONS ]. Pregnancy Neonatal Opioid Withdrawal Syndrome Inform female patients of reproductive potential that prolonged use of hydrocodone bitartrate and acetaminophen tablets during pregnancy can result in neonatal opioid withdrawal syndrome, which may be life-threatening if not recognized and treated [see WARNINGS , PRECAUTIONS ; Pregnancy ]. Embryo-Fetal Toxicity Inform female patients of reproductive potential that hydrocodone bitartrate and acetaminophen tablets can cause fetal harm and to inform their healthcare provider of a known or suspected pregnancy [see PRECAUTIONS ; Pregnancy ]. Lactation Advise nursing mothers to monitor infants for increased sleepiness (more than usual), breathing difficulties, or limpness. Instruct nursing mothers to seek immediate medical care if they notice these signs [see PRECAUTIONS ; Nursing Mothers ]. Infertility Inform patients that chronic use of opioids may cause reduced fertility. It is not known whether these effects on fertility are reversible [see ADVERSE REACTIONS ]. Driving or Operating Heavy Machinery Inform patients that hydrocodone bitartrate and acetaminophen tablets may impair the ability to perform potentially hazardous activities such as driving a car or operating heavy machinery. Advise patients not to perform such tasks until they know how they will react to the medication [see WARNINGS ]. Constipation Advise patients of the potential for severe constipation, including management instructions and when to seek medical attention [see ADVERSE REACTIONS , CLINICAL PHARMACOLOGY ]."
      ],
      "laboratory_tests": [
        "Laboratory Tests In patients with severe hepatic or renal disease, effects of therapy should be followed with serial liver and/or renal function tests."
      ],
      "drug_interactions": [
        "Drug Interactions Inhibitors of CYP3A4 and CYP2D6 The concomitant use of hydrocodone bitartrate and acetaminophen tablets and CYP3A4 inhibitors, such as macrolide antibiotics (e.g., erythromycin), azole-antifungal agents (e.g. ketoconazole), and protease inhibitors (e.g., ritonavir), can increase the plasma concentration of the hydrocodone from hydrocodone bitartrate and acetaminophen tablets, resulting in increased or prolonged opioid effects. These effects could be more pronounced with concomitant use of hydrocodone bitartrate and acetaminophen tablets and both CYP3A4 and CYP2D6 inhibitors, particularly when an inhibitor is added after a stable dose of hydrocodone bitartrate and acetaminophen tablets is achieved [see WARNINGS ]. After stopping a CYP3A4 inhibitor, as the effects of the inhibitor decline, the hydrocodone plasma concentration will decrease [see CLINICAL PHARMACOLOGY ], resulting in decreased opioid efficacy or a withdrawal syndrome in patients who had developed physical dependence to hydrocodone bitartrate and acetaminophen tablets. If concomitant use is necessary, consider dosage reduction of hydrocodone bitartrate and acetaminophen tablets until stable drug effects are achieved. Follow patients for respiratory depression and sedation at frequent intervals. If a CYP3A4 inhibitor is discontinued, consider increasing the hydrocodone bitartrate and acetaminophen tablets dosage until stable drug effects are achieved. Follow for signs or symptoms of opioid withdrawal. Inducers of CYP3A4 The concomitant use of hydrocodone bitartrate and acetaminophen tablets and CYP3A4 inducers, such as rifampin, carbamazepine, and phenytoin, can decrease the plasma concentration of hydrocodone [see CLINICAL PHARMACOLOGY ], resulting in decreased efficacy or onset of a withdrawal syndrome in patients who have developed physical dependence to hydrocodone [see WARNINGS ]. After stopping a CYP3A4 inducer, as the effects of the inducer decline, the hydrocodone plasma concentration will increase [see CLINICAL PHARMACOLOGY ], which could increase or prolong both the therapeutic effects and adverse reactions, and may cause serious respiratory depression. If concomitant use is necessary, consider increasing the hydrocodone bitartrate and acetaminophen tablets dosage until stable drug effects are achieved. Follow the patient for signs and symptoms of opioid withdrawal. If a CYP3A4 inducer is discontinued, consider hydrocodone bitartrate and acetaminophen tablets dosage reduction and follow for signs of respiratory depression. Benzodiazepines and Other CNS Depressants Due to additive pharmacologic effect, the concomitant use of benzodiazepines and other CNS depressants, such as benzodiazepines and other sedative hypnotics, anxiolytics, and tranquilizers, muscle relaxants, general anesthetics, antipsychotics, and other opioids, including alcohol, can increase the risk of hypotension, respiratory depression, profound sedation, coma, and death. Reserve concomitant prescribing of these drugs for use in patients for whom alternative treatment options are inadequate. Limit dosages and durations to the minimum required. Follow patients closely for signs of respiratory depression and sedation. If concomitant use is warranted, consider prescribing naloxone for the emergency treatment of opioid overdose[see WARNINGS ]. Serotonergic Drugs The concomitant use of opioids with other drugs that affect the serotonergic neurotransmitter system, such as selective serotonin reuptake inhibitors (SSRIs), serotonin and norepinephrine reuptake inhibitors (SNRIs), tricyclic antidepressants (TCAs), triptans, 5-HT3 receptor antagonists, drugs that affect the serotonin neurotransmitter system (e.g., mirtazapine, trazodone, tramadol), certain muscle relaxants (i.e., cyclobenzaprine,metaxalone) and monoamine oxidase (MAO) inhibitors (those intended to treat psychiatric disorders and also others, such as linezolid and intravenous methylene blue), has resulted in serotonin syndrome [see PRECAUTIONS ; Information for Patients ]. If concomitant use is warranted, carefully follow the patient, particularly during treatment initiation and dose adjustment. Discontinue hydrocodone bitartrate and acetaminophen tablets if serotonin syndrome is suspected. Monoamine Oxidase Inhibitors (MAOIs) The concomitant use of opioids and MAOIs, such as phenelzine, tranylcypromine, or linezolid, may manifest as serotonin syndrome, or opioid toxicity (e.g., respiratory depression, coma) [see WARNINGS ]. The use of hydrocodone bitartrate and acetaminophen tablets is not recommended for patients taking MAOIs or within 14 days of stopping such treatment. If urgent use of an opioid is necessary, use test doses and frequent titration of small doses to treat pain while closely monitoring blood pressure and signs and symptoms of CNS and respiratory depression. Mixed Agonist/Antagonist and Partial Agonist Opioid Analgesics The concomitant use of opioids with other opioid analgesics, such as butorphanol, nalbuphine, pentazocine, may reduce the analgesic effect of hydrocodone bitartrate and acetaminophen tablets and/or precipitate withdrawal symptoms. Advise patient to avoid concomitant use of these drugs. Muscle Relaxants Hydrocodone bitartrate and acetaminophen tablets may enhance the neuromuscular blocking action of skeletal muscle relaxants and produce an increased degree of respiratory depression. If concomitant use is warranted, monitor patients for signs of respiratory depression that may be greater than otherwise expected and decrease the dosage of hydrocodone bitartrate and acetaminophen tablets and/or the muscle relaxant as necessary. Due to the risk of respiratory depression with concomitant use of skeletal muscle relaxants and opioids, consider prescribing naloxone for the emergency treatment of opioid overdose [see WARNINGS ]. Diuretics Opioids can reduce the efficacy of diuretics by inducing the release of antidiuretic hormone. If concomitant use is warranted, follow patients for signs of diminished diuresis and/or effects on blood pressure and increase the dosage of the diuretic as needed. Anticholinergic Drugs The concomitant use of anticholinergic drugs may increase risk of urinary retention and/or severe constipation, which may lead to paralytic ileus. If concomitant use is warranted, follow patients for signs and symptoms of urinary retention or reduced gastric motility when hydrocodone bitartrate and acetaminophen tablets are used concomitantly with anticholinergic drugs."
      ],
      "drug_and_or_laboratory_test_interactions": [
        "Drug/Laboratory Test Interactions Acetaminophen may produce false-positive test results for urinary 5-hydroxyindoleacetic acid."
      ],
      "carcinogenesis_and_mutagenesis_and_impairment_of_fertility": [
        "Carcinogenesis, Mutagenesis, Impairment of Fertility Carcinogenesis Long-term studies to evaluate the carcinogenic potential of the combination of hydrocodone bitartrate and acetaminophen tablets have not been conducted. Long-term studies in mice and rats have been completed by the National Toxicology Program to evaluate the carcinogenic potential of acetaminophen. In 2-year feeding studies, F344/N rats and B6C3F1 mice were fed a diet containing acetaminophen up to 6000 ppm. Female rats demonstrated equivocal evidence of carcinogenic activity based on increased incidences of mononuclear cell leukemia at 0.8 times the maximum human daily dose (MHDD) of 4 grams/day, based on a body surface area comparison. In contrast, there was no evidence of carcinogenic activity in male rats that received up to 0.7 times or mice at up to 1.2-1.4 times the MHDD, based on a body surface area comparison. Mutagenesis In the published literature, acetaminophen has been reported to be clastogenic when administered at 1500 mg/kg/day to the rat model (3.6-times the MHDD, based on a body surface area comparison). In contrast, no clastogenicity was noted at a dose of 750 mg/kg/day (1.8-times the MHDD, based on a body surface area comparison), suggesting a threshold effect. Impairment of Fertility In studies conducted by the National Toxicology Program, fertility assessments with acetaminophen have been completed in Swiss CD-1 mice via a continuous breeding study. There were no effects on fertility parameters in mice consuming up to 1.7 times the MHDD of acetaminophen, based on a body surface area comparison. Although there was no effect on sperm motility or sperm density in the epididymis, there was a significant increase in the percentage of abnormal sperm in mice consuming 1.78 times the MHDD (based on a body surface comparison) and there was a reduction in the number of mating pairs producing a fifth litter at this dose, suggesting the potential for cumulative toxicity with chronic administration of acetaminophen near the upper limit of daily dosing. Published studies in rodents report that oral acetaminophen treatment of male animals at doses that are 1.2 times the MHDD and greater (based on a body surface comparison) result in decreased testicular weights, reduced spermatogenesis, reduced fertility, and reduced implantation sites in females given the same doses. These effects appear to increase with the duration of treatment. The clinical significance of these findings is not known. Infertility Chronic use of opioids may cause reduced fertility in females and males of reproductive potential. It is not known whether these effects on fertility are reversible [see ADVERSE REACTIONS ]."
      ],
      "pregnancy": [
        "Pregnancy Teratogenic Effects There are no adequate and well-controlled studies in pregnant women. Hydrocodone bitartrate and acetaminophen tablets should be used during pregnancy only if the potential benefit justifies the potential risk to the fetus. Nonteratogenic Effects Fetal/Neonatal Adverse Reactions Prolonged use of opioid analgesics during pregnancy for medical or nonmedical purposes can result in physical dependence in the neonate and neonatal opioid withdrawal syndrome shortly after birth. Neonatal opioid withdrawal syndrome presents as irritability, hyperactivity, abnormal sleep pattern, high pitched cry, tremor, vomiting, diarrhea and failure to gain weight. The onset, duration, and severity of neonatal opioid withdrawal syndrome vary based on the specific opioid used, duration of use, timing and amount of last maternal use, and rate of elimination of the drug by the newborn. Observe newborns for symptoms of neonatal opioid withdrawal syndrome and manage accordingly [see WARNINGS ]."
      ],
      "labor_and_delivery": [
        "Labor or Delivery Opioids cross the placenta and may produce respiratory depression and psycho-physiologic effects in neonates. An opioid antagonist, such as naloxone, must be available for reversal of opioid-induced respiratory depression in the neonate. Hydrocodone bitartrate and acetaminophen tablets are not recommended for use in pregnant women during or immediately prior to labor, when other analgesic techniques are more appropriate. Opioid analgesics, including hydrocodone bitartrate and acetaminophen tablets, can prolong labor through actions which temporarily reduce the strength, duration, and frequency of uterine contractions. However, this effect is not consistent and may be offset by an increased rate of cervical dilation, which tends to shorten labor. Monitor neonates exposed to opioid analgesics during labor for signs of excess sedation and respiratory depression."
      ],
      "nursing_mothers": [
        "Nursing Mothers Hydrocodone is present in human milk. The developmental and health benefits of breastfeeding should be considered along with the mother\u2019s clinical need for hydrocodone bitartrate and acetaminophen tablets and any potential adverse effects on the breastfed infant from hydrocodone bitartrate and acetaminophen tablets or from the underlying maternal condition. Infants exposed to hydrocodone bitartrate and acetaminophen tablets through breast milk should be monitored for excess sedation and respiratory depression. Withdrawal symptoms can occur in breastfed infants when maternal administration of an opioid analgesic is stopped, or when breast-feeding is stopped."
      ],
      "pediatric_use": [
        "Pediatric Use Safety and effectiveness of hydrocodone bitartrate and acetaminophen tablets in pediatric patients have not been established."
      ],
      "geriatric_use": [
        "Geriatric Use Elderly patients (aged 65 years or older) may have increased sensitivity to hydrocodone bitartrate and acetaminophen tablets. In general, use caution when selecting a dosage for an elderly patient, usually starting at the low end of the dosing range, reflecting the greater frequency of decreased hepatic, renal, or cardiac function, and of concomitant disease or other drug therapy. Respiratory depression is the chief risk for elderly patients treated with opioids, and has occurred after large initial doses were administered to patients who were not opioid-tolerant or when opioids were co-administered with other agents that depress respiration. Titrate the dosage of hydrocodone bitartrate and acetaminophen tablets slowly in geriatric patients and follow closely for signs of central nervous system and respiratory depression [see WARNINGS ]. Hydrocodone and acetaminophen are known to be substantially excreted by the kidney, and the risk of adverse reactions to this drug may be greater in patients with impaired renal function. Because elderly patients are more likely to have decreased renal function, care should be taken in dose selection, and it may be useful to monitor renal function."
      ],
      "adverse_reactions": [
        "ADVERSE REACTIONS The following adverse reactions have been identified during post approval use of hydrocodone and acetaminophen tablets. Because these reactions are reported voluntarily from a population of uncertain size, it is not always possible to reliably estimate their frequency or establish a causal relationship to drug exposure. The most frequently reported adverse reactions are light-headedness, dizziness, sedation, nausea and vomiting. Other adverse reactions include: Central Nervous System: Drowsiness, mental clouding, lethargy, impairment of mental and physical performance, anxiety, fear, dysphoria, psychological dependence, mood changes. Gastrointestinal System: Constipation. Genitourinary System: Ureteral spasm, spasm of vesical sphincters, and urinary retention. Special Senses: Cases of hearing impairment or permanent loss have been reported predominately in patients with chronic overdose. Dermatological: Skin rash, pruritus, Stevens-Johnson syndrome, toxic epidermal necrolysis, allergic reactions. Hematological: Thrombocytopenia, agranulocytosis. Serotonin syndrome: Cases of serotonin syndrome, a potentially life-threatening condition, have been reported during concomitant use of opioids with serotonergic drugs. Adrenal insufficiency: Cases of adrenal insufficiency have been reported with opioid use, more often following greater than one month of use. Anaphylaxis: Anaphylaxis has been reported with ingredients contained in hydrocodone bitartrate and acetaminophen tablets. Androgen deficiency: Cases of androgen deficiency have occurred with chronic use of opioids [see CLINICAL PHARMACOLOGY ]."
      ],
      "drug_abuse_and_dependence": [
        "DRUG ABUSE AND DEPENDENCE CONTROLLED SUBSTANCE Hydrocodone bitartrate and acetaminophen tablets contain hydrocodone, a Schedule II controlled substance. ABUSE Hydrocodone bitartrate and acetaminophen tablets contain hydrocodone, a substance with a high potential for abuse similar to other opioids, including fentanyl, hydrocodone, hydromorphone, methadone, morphine, oxycodone, oxymorphone, and tapentadol, can be abused and are subject to misuse, addiction, and criminal diversion [see WARNINGS ]. All patients treated with opioids require careful monitoring for signs of abuse and addiction, because use of opioid analgesic products carries the risk of addiction even under appropriate medical use. Prescription drug abuse is the intentional non-therapeutic use of a prescription drug, even once, for its rewarding psychological or physiological effects. Drug addiction is a cluster of behavioral, cognitive, and physiological phenomena that develop after repeated substance use and includes a strong desire to take the drug, difficulties in controlling its use, persisting in its use despite harmful consequences, a higher priority given to drug use than to other activities and obligations, increased tolerance, and sometimes a physical withdrawal. \u201cDrug-seeking\u201d behavior is very common in persons with substance use disorders. Drug-seeking tactics include emergency calls or visits near the end of office hours, refusal to undergo appropriate examination, testing, or referral, repeated \u201closs\u201d of prescriptions, tampering with prescriptions and reluctance to provide prior medical records or contact information for other treating healthcare provider(s). \u201cDoctor shopping\u201d (visiting multiple prescribers to obtain additional prescriptions) is common among drug abusers and people suffering from untreated addiction. Preoccupation with achieving adequate pain relief can be appropriate behavior in a patient with poor pain control. Abuse and addiction are separate and distinct from physical dependence and tolerance. Health care providers should be aware that addiction may not be accompanied by concurrent tolerance and symptoms of physical dependence in all addicts. In addition, abuse of opioids can occur in the absence of true addiction. Hydrocodone bitartrate and acetaminophen tablets, like other opioids, can be diverted for non-medical use into illicit channels of distribution. Careful record-keeping of prescribing information, including quantity, frequency, and renewal requests, as required by state and federal law, is strongly advised. Proper assessment of the patient, proper prescribing practices, periodic re-evaluation of therapy, and proper dispensing and storage are appropriate measures that help to limit abuse of opioid drugs. Risks Specific to Abuse of Hydrocodone Bitartrate and Acetaminophen Tablets Hydrocodone bitartrate and acetaminophen tablets are for oral use only. Hydrocodone bitartrate and acetaminophen tablets pose a risk of overdose and death. The risk is increased with concurrent abuse of hydrocodone bitartrate and acetaminophen tablets with alcohol and other central nervous system depressants. Parenteral drug abuse is commonly associated with transmission of infectious diseases such as hepatitis and HIV. DEPENDENCE Both tolerance and physical dependence can develop during chronic opioid therapy. Tolerance is the need for increasing doses of opioids to maintain a defined effect such as analgesia (in the absence of disease progression or other external factors). Tolerance may occur to both the desired and undesired effects of drugs, and may develop at different rates for different effects. Physical dependence is a physiological state in which the body adapts to the drug after a period of regular exposure, resulting in withdrawal symptoms after abrupt discontinuation or a significant dosage reduction of a drug. Withdrawal also may be precipitated through the administration of drugs with opioid antagonist activity (e.g., naloxone, nalmefene), mixed agonist/antagonist analgesics (e.g., pentazocine, butorphanol, nalbuphine), or partial agonists (e.g., buprenorphine). Physical dependence may not occur to a clinically significant degree until after several days to weeks of continued opioid usage. Do not abruptly discontinue Hydrocodone Bitartrate and Acetaminophen Tablets in a patient physically dependent on opioids. Rapid tapering of Hydrocodone Bitartrate and Acetaminophen Tablets in a patient physically dependent on opioids may lead to serious withdrawal symptoms, uncontrolled pain, and suicide. Rapid discontinuation has also been associated with attempts to find other sources of opioid analgesics, which may be confused with drug-seeking for abuse. When discontinuing Hydrocodone Bitartrate and Acetaminophen Tablets, gradually taper the dosage using a patient specific plan that considers the following: the dose of Hydrocodone Bitartrate and Acetaminophen Tablets the patient has been taking, the duration of treatment, and the physical and psychological attributes of the patient. To improve the likelihood of a successful taper and minimize withdrawal symptoms, it is important that the opioid tapering schedule is agreed upon by the patient. In patients taking opioids for a long duration at high doses, ensure that a multimodal approach to pain management, including mental health support (if needed), is in place prior to initiating an opioid analgesic taper [see DOSAGE AND ADMINISTRATION , WARNINGS ]. Infants born to mothers physically dependent on opioids will also be physically dependent and may exhibit respiratory difficulties and withdrawal signs [see PRECAUTIONS ; Pregnancy ]."
      ],
      "controlled_substance": [
        "CONTROLLED SUBSTANCE Hydrocodone bitartrate and acetaminophen tablets contain hydrocodone, a Schedule II controlled substance."
      ],
      "abuse": [
        "ABUSE Hydrocodone bitartrate and acetaminophen tablets contain hydrocodone, a substance with a high potential for abuse similar to other opioids, including fentanyl, hydrocodone, hydromorphone, methadone, morphine, oxycodone, oxymorphone, and tapentadol, can be abused and are subject to misuse, addiction, and criminal diversion [see WARNINGS ]. All patients treated with opioids require careful monitoring for signs of abuse and addiction, because use of opioid analgesic products carries the risk of addiction even under appropriate medical use. Prescription drug abuse is the intentional non-therapeutic use of a prescription drug, even once, for its rewarding psychological or physiological effects. Drug addiction is a cluster of behavioral, cognitive, and physiological phenomena that develop after repeated substance use and includes a strong desire to take the drug, difficulties in controlling its use, persisting in its use despite harmful consequences, a higher priority given to drug use than to other activities and obligations, increased tolerance, and sometimes a physical withdrawal. \u201cDrug-seeking\u201d behavior is very common in persons with substance use disorders. Drug-seeking tactics include emergency calls or visits near the end of office hours, refusal to undergo appropriate examination, testing, or referral, repeated \u201closs\u201d of prescriptions, tampering with prescriptions and reluctance to provide prior medical records or contact information for other treating healthcare provider(s). \u201cDoctor shopping\u201d (visiting multiple prescribers to obtain additional prescriptions) is common among drug abusers and people suffering from untreated addiction. Preoccupation with achieving adequate pain relief can be appropriate behavior in a patient with poor pain control. Abuse and addiction are separate and distinct from physical dependence and tolerance. Health care providers should be aware that addiction may not be accompanied by concurrent tolerance and symptoms of physical dependence in all addicts. In addition, abuse of opioids can occur in the absence of true addiction. Hydrocodone bitartrate and acetaminophen tablets, like other opioids, can be diverted for non-medical use into illicit channels of distribution. Careful record-keeping of prescribing information, including quantity, frequency, and renewal requests, as required by state and federal law, is strongly advised. Proper assessment of the patient, proper prescribing practices, periodic re-evaluation of therapy, and proper dispensing and storage are appropriate measures that help to limit abuse of opioid drugs. Risks Specific to Abuse of Hydrocodone Bitartrate and Acetaminophen Tablets Hydrocodone bitartrate and acetaminophen tablets are for oral use only. Hydrocodone bitartrate and acetaminophen tablets pose a risk of overdose and death. The risk is increased with concurrent abuse of hydrocodone bitartrate and acetaminophen tablets with alcohol and other central nervous system depressants. Parenteral drug abuse is commonly associated with transmission of infectious diseases such as hepatitis and HIV."
      ],
      "dependence": [
        "DEPENDENCE Both tolerance and physical dependence can develop during chronic opioid therapy. Tolerance is the need for increasing doses of opioids to maintain a defined effect such as analgesia (in the absence of disease progression or other external factors). Tolerance may occur to both the desired and undesired effects of drugs, and may develop at different rates for different effects. Physical dependence is a physiological state in which the body adapts to the drug after a period of regular exposure, resulting in withdrawal symptoms after abrupt discontinuation or a significant dosage reduction of a drug. Withdrawal also may be precipitated through the administration of drugs with opioid antagonist activity (e.g., naloxone, nalmefene), mixed agonist/antagonist analgesics (e.g., pentazocine, butorphanol, nalbuphine), or partial agonists (e.g., buprenorphine). Physical dependence may not occur to a clinically significant degree until after several days to weeks of continued opioid usage. Do not abruptly discontinue Hydrocodone Bitartrate and Acetaminophen Tablets in a patient physically dependent on opioids. Rapid tapering of Hydrocodone Bitartrate and Acetaminophen Tablets in a patient physically dependent on opioids may lead to serious withdrawal symptoms, uncontrolled pain, and suicide. Rapid discontinuation has also been associated with attempts to find other sources of opioid analgesics, which may be confused with drug-seeking for abuse. When discontinuing Hydrocodone Bitartrate and Acetaminophen Tablets, gradually taper the dosage using a patient specific plan that considers the following: the dose of Hydrocodone Bitartrate and Acetaminophen Tablets the patient has been taking, the duration of treatment, and the physical and psychological attributes of the patient. To improve the likelihood of a successful taper and minimize withdrawal symptoms, it is important that the opioid tapering schedule is agreed upon by the patient. In patients taking opioids for a long duration at high doses, ensure that a multimodal approach to pain management, including mental health support (if needed), is in place prior to initiating an opioid analgesic taper [see DOSAGE AND ADMINISTRATION , WARNINGS ]. Infants born to mothers physically dependent on opioids will also be physically dependent and may exhibit respiratory difficulties and withdrawal signs [see PRECAUTIONS ; Pregnancy ]."
      ],
      "overdosage": [
        "OVERDOSAGE Following an acute overdosage, toxicity may result from hydrocodone or acetaminophen. Clinical Presentation Acute overdosage with hydrocodone bitartrate and acetaminophen tablets can be manifested by respiratory depression, somnolence progressing to stupor or coma, skeletal muscle flaccidity, cold and clammy skin, constricted pupils, and, in some cases, pulmonary edema, bradycardia, hypotension, partial or complete airway obstruction, atypical snoring, and death. Marked mydriasis rather than miosis may be seen with hypoxia in overdose situations. Acetaminophen Dose-dependent, potentially fatal hepatic necrosis is the most serious adverse effect of acetaminophen overdosage. Renal tubular necrosis, hypoglycemic coma and coagulation defects may also occur. Early symptoms following a potentially hepatotoxic overdose may include: nausea, vomiting, diaphoresis and general malaise. Clinical and laboratory evidence of hepatic toxicity may not be apparent until 48 to 72 hours post-ingestion. Treatment of Overdose Hydrocodone In case of overdose, priorities are the re-establishment of a patent and protected airway and institution of assisted or controlled ventilation, if needed. Employ other supportive measures (including oxygen and vasopressors) in the management of circulatory shock and pulmonary edema as indicated. Cardiac arrest or arrhythmias will require advanced life-support techniques. Opioid antagonists, such as naloxone are specific antidotes to respiratory depression resulting from opioid overdose. For clinically significant respiratory or circulatory depression secondary to opioid overdose, administer an opioid antagonist. Because the duration of opioid reversal is expected to be less than the duration of action of hydrocodone in hydrocodone bitartrate and acetaminophen tablets, carefully monitor the patient until spontaneous respiration is reliably reestablished. If the response to an opioid antagonist is suboptimal or only brief in nature, administer additional antagonist as directed by the product\u2019s prescribing information. In an individual physically dependent on opioids, administration of the recommended usual dosage of the antagonist will precipitate an acute withdrawal syndrome. The severity of the withdrawal symptoms experienced will depend on the degree of physical dependence and the dose of the antagonist administered. If a decision is made to treat serious respiratory depression in the physically dependent patient, administration of the antagonist should be initiated with care and by titration with smaller than usual doses of the antagonist. Acetaminophen Gastric decontamination with activated charcoal should be administered just prior to N-acetylcysteine (NAC) to decrease systemic absorption if acetaminophen ingestion is known or suspected to have occurred within a few hours of presentation. Serum acetaminophen levels should be obtained immediately if the patient presents 4 hours or more after ingestion to assess potential risk of hepatotoxicity; acetaminophen levels drawn less than 4 hours post-ingestion may be misleading. To obtain the best possible outcome, NAC should be administered as soon as possible where impending or evolving liver injury is suspected. Intravenous NAC may be administered when circumstances preclude oral administration. Vigorous supportive therapy is required in severe intoxication. Procedures to limit the continuing absorption of the drug must be readily performed since the hepatic injury is dose dependent and occurs early in the course of intoxication."
      ],
      "dosage_and_administration": [
        "DOSAGE AND ADMINISTRATION Important Dosage and Administration Instructions Use the lowest effective dosage for the shortest duration consistent with individual patient treatment goals [see WARNINGS ]. Initiate the dosing regimen for each patient individually, taking into account the patient's severity of pain, patient response, prior analgesic treatment experience, and risk factors for addiction, abuse, and misuse [see WARNINGS ]. Follow patients closely for respiratory depression, especially within the first 24-72 hours of initiating therapy and following dosage increases with hydrocodone bitartrate and acetaminophen tablets and adjust the dosage accordingly [see WARNINGS ]. Patient Access to Naloxone for the Emergency Treatment of Opioid Overdose Discuss the availability of naloxone for the emergency treatment of opioid overdose with the patient and caregiver and assess the potential need for access to naloxone, both when initiating and renewing treatment with hydrocodone bitartrate and acetaminophen tablets [see WARNINGS, Life-Threatening Respiratory Depression ; PRECAUTIONS , Information for Patients ]. Inform patients and caregivers about the various ways to obtain naloxone as permitted by individual state naloxone dispensing and prescribing regulations (e.g., by prescription, directly from a pharmacist, or as part of a community-based program). Consider prescribing naloxone, based on the patient\u2019s risk factors for overdose, such as concomitant use of CNS depressants, a history of opioid use disorder, or prior opioid overdose. The presence of risk factors for overdose should not prevent the proper management of pain in any given patient [see WARNINGS, Addiction, Abuse, and Misuse, Life-Threatening Respiratory Depression, Risks from Concomitant Use with Benzodiazepines or Other CNS Depressants ]. Consider prescribing naloxone when the patient has household members (including children) or other close contacts at risk for accidental ingestion or overdose. Initial Dosage Initiating Treatment with Hydrocodone Bitartrate and Acetaminophen Tablets 5 mg/300 mg The usual adult dosage is one or two tablets every four to six hours as needed for pain. The total daily dosage should not exceed 8 tablets. 7.5 mg/300 mg 10 mg/300 mg The usual adult dosage is one tablet every four to six hours as needed for pain. The total daily dosage should not exceed 6 tablets. Conversion from Other Opioids to Hydrocodone Bitartrate and Acetaminophen Tablets There is inter-patient variability in the potency of opioid drugs and opioid formulations. Therefore, a conservative approach is advised when determining the total daily dosage of hydrocodone bitartrate and acetaminophen tablets. It is safer to underestimate a patient\u2019s 24-hour hydrocodone bitartrate and acetaminophen tablets dosage than to overestimate the 24-hour hydrocodone bitartrate and acetaminophen tablets dosage and manage an adverse reaction due to overdose. Conversion from Hydrocodone Bitartrate and Acetaminophen Tablets to Extended-Release Hydrocodone The relative bioavailability of hydrocodone from hydrocodone bitartrate and acetaminophen tablets compared to extended-release hydrocodone products is unknown, so conversion to extended-release products must be accompanied by close observation for signs of excessive sedation and respiratory depression. Titration and Maintenance of Therapy Individually titrate hydrocodone bitartrate and acetaminophen tablets to a dose that provides adequate analgesia and minimizes adverse reactions. Continually reevaluate patients receiving hydrocodone bitartrate and acetaminophen tablets to assess the maintenance of pain control and the relative incidence of adverse reactions, as well as monitoring for the development of addiction, abuse, or misuse [see WARNINGS ] . Frequent communication is important among the prescriber, other members of the healthcare team, the patient, and the caregiver/family during periods of changing analgesic requirements, including initial titration. If the level of pain increases after dosage stabilization, attempt to identify the source of increased pain before increasing the hydrocodone bitartrate and acetaminophen tablets dosage. If unacceptable opioid related adverse reactions are observed, consider reducing the dosage. Adjust the dosage to obtain an appropriate balance between management of pain and opioid-related adverse reactions. Safe Reduction or Discontinuation of Hydrocodone Bitartrate and Acetaminophen Tablets Do not abruptly discontinue Hydrocodone Bitartrate and Acetaminophen Tablets in patients who may be physically dependent on opioids. Rapid discontinuation of opioid analgesics in patients who are physically dependent on opioids has resulted in serious withdrawal symptoms, uncontrolled pain, and suicide. Rapid discontinuation has also been associated with attempts to find other sources of opioid analgesics, which may be confused with drug-seeking for abuse. Patients may also attempt to treat their pain or withdrawal symptoms with illicit opioids, such as heroin, and other substances. When a decision has been made to decrease the dose or discontinue therapy in an opioid-dependent patient taking Hydrocodone Bitartrate and Acetaminophen Tablets, there are a variety of factors that should be considered, including the dose of Hydrocodone Bitartrate and Acetaminophen Tablets the patient has been taking, the duration of treatment, the type of pain being treated, and the physical and psychological attributes of the patient. It is important to ensure ongoing care of the patient and to agree on an appropriate tapering schedule and follow-up plan so that patient and provider goals and expectations are clear and realistic. When opioid analgesics are being discontinued due to a suspected substance use disorder, evaluate and treat the patient, or refer for evaluation and treatment of the substance use disorder. Treatment should include evidence-based approaches, such as medication assisted treatment of opioid use disorder. Complex patients with co-morbid pain and substance use disorders may benefit from referral to a specialist. There are no standard opioid tapering schedules that are suitable for all patients. Good clinical practice dictates a patient-specific plan to taper the dose of the opioid gradually. For patients on Hydrocodone Bitartrate and Acetaminophen Tablets who are physically opioid-dependent, initiate the taper by a small enough increment (e.g., no greater than 10% to 25% of the total daily dose) to avoid withdrawal symptoms, and proceed with dose lowering at an interval of every 2 to 4 weeks. Patients who have been taking opioids for briefer periods of time may tolerate a more rapid taper. It may be necessary to provide the patient with lower dosage strengths to accomplish a successful taper. Reassess the patient frequently to manage pain and withdrawal symptoms, should they emerge. Common withdrawal symptoms include restlessness, lacrimation, rhinorrhea, yawning, perspiration, chills, myalgia, and mydriasis. Other signs and symptoms also may develop, including irritability, anxiety, backache, joint pain, weakness, abdominal cramps, insomnia, nausea, anorexia, vomiting, diarrhea, or increased blood pressure, respiratory rate, or heart rate. If withdrawal symptoms arise, it may be necessary to pause the taper for a period of time or raise the dose of the opioid analgesic to the previous dose, and then proceed with a slower taper. In addition, monitor patients for any changes in mood, emergence of suicidal thoughts, or use of other substances. When managing patients taking opioid analgesics, particularly those who have been treated for a long duration and/or with high doses for chronic pain, ensure that a multimodal approach to pain management, including mental health support (if needed), is in place prior to initiating an opioid analgesic taper. A multimodal approach to pain management may optimize the treatment of chronic pain, as well as assist with the successful tapering of the opioid analgesic [see WARNINGS/ Withdrawal , DRUG ABUSE AND DEPENDENCE ]."
      ],
      "dosage_and_administration_table": [
        "<table cellspacing=\"0\" cellpadding=\"0\" border=\"0\" width=\"829.92\"><colgroup><col width=\"20.1923076923077%\"/><col width=\"79.8076923076923%\"/></colgroup><tbody><tr styleCode=\"Botrule\"><td styleCode=\"Lrule Rrule\" valign=\"top\">  <content styleCode=\"underline\"><content styleCode=\"bold\">5 mg/300 mg</content></content>   </td><td styleCode=\"Rrule\" valign=\"top\">  <content styleCode=\"underline\">The usual adult dosage is one or two tablets every four to six hours as needed for pain. The total daily dosage should not exceed 8 tablets.</content>   </td></tr><tr><td styleCode=\"Lrule Rrule\" valign=\"top\">  <content styleCode=\"bold\"><content styleCode=\"underline\">7.5 mg/300 mg </content></content>   <content styleCode=\"bold\"><content styleCode=\"underline\">10 mg/300 mg </content></content>   </td><td styleCode=\"Rrule\" valign=\"top\">  <content styleCode=\"underline\">The usual adult dosage is one tablet every four to six hours as needed for pain. The total daily dosage should not exceed 6 tablets. </content>   </td></tr></tbody></table>"
      ],
      "how_supplied": [
        "HOW SUPPLIED Hydrocodone Bitartrate and Acetaminophen Tablets USP are supplied as follows: Hydrocodone Bitartrate and Acetaminophen Tablets USP, 5 mg/300 mg are white to off-white, uncoated, capsule shaped tablets, debossed with \u201cU114\u201d on one side and \u201cbreakline\u201d on the other side. Bottles of 30 NDC 13107-211-30 Bottles of 100 NDC 13107-211-01 Bottles of 500 NDC 13107-211-05 Bottles of 1,000 NDC 13107-211-99 Hydrocodone Bitartrate and Acetaminophen Tablets USP, 7.5 mg/300 mg are white to off-white, uncoated, oval shaped tablets, debossed with \u201cU115\u201d on one side and \u201cbreakline\u201d on the other side. Bottles of 30 NDC 13107-212-30 Bottles of 100 NDC 13107-212-01 Bottles of 500 NDC 13107-212-05 Bottles of 1,000 NDC 13107-212-99 Hydrocodone Bitartrate and Acetaminophen Tablets USP, 10 mg/300 mg are white to off-white, uncoated, capsule shaped tablets, debossed with \u201cU116\u201d on one side and \u201cbreakline\u201d on the other side. Bottles of 30 NDC 13107-213-30 Bottles of 100 NDC 13107-213-01 Bottles of 500 NDC 13107-213-05 Bottles of 1,000 NDC 13107-213-99 STORAGE: Store at 20\u00b0 to 25\u00b0C (68\u00b0 to 77\u00b0F) [see USP Controlled Room Temperature]. Store Hydrocodone Bitartrate and Acetaminophen Tablets securely and dispose of properly [see PRECAUTIONS / Information for Patients ]. Dispense in a tight, light resistant container as defined in the USP with a child-resistant closure A Schedule II Narcotic Dispense with Medication Guide available at https://www.aurobindousa.com/medication-guides/ Distributed by: Aurobindo Pharma USA, Inc 279 Princeton-Hightstown Road East Windsor, NJ 08520 Revised: 08/2020"
      ],
      "spl_medguide": [
        "MEDICATION GUIDE Hydrocodone Bitartrate ( hye\" droe koe' done bye tar trate ) and Acetaminophen ( a seet\"a min' oh fen ) Tablets USP, CII Hydrocodone Bitartrate and Acetaminophen Tablets are: A strong prescription pain medicine that contains an opioid (narcotic) that is used to manage pain severe enough to require an opioid pain medicine, when other pain treatments such as non-opioid pain medicines do not treat your pain well enough or you cannot tolerate them. An opioid pain medicine that can put you at risk for overdose and death. Even if you take your dose correctly as prescribed you are at risk for opioid addiction, abuse, and misuse that can lead to death. Important information about Hydrocodone Bitartrate and Acetaminophen Tablets : Get emergency help or call 911 right away if you take too much Hydrocodone Bitartrate and Acetaminophen Tablets (overdose). When you first start taking Hydrocodone Bitartrate and Acetaminophen Tablets, when your dose is changed, or if you take too much (overdose), serious or life-threatening breathing problems that can lead to death may occur. Talk to your healthcare provider about naloxone, a medicine for the emergency treatment of an opioid overdose. Taking Hydrocodone Bitartrate and Acetaminophen Tablets with other opioid medicines, benzodiazepines, alcohol, or other central nervous system depressants (including street drugs) can cause severe drowsiness, decreased awareness, breathing problems, coma, and death. Never give anyone else your Hydrocodone Bitartrate and Acetaminophen Tablets. They could die from taking it. Selling or giving away Hydrocodone Bitartrate and Acetaminophen Tablets are against the law. Store Hydrocodone Bitartrate and Acetaminophen Tablets securely, out of sight and reach of children, and in a location not accessible by others, including visitors to the home. Do not take Hydrocodone Bitartrate and Acetaminophen Tablets if you have: severe asthma, trouble breathing, or other lung problems. a bowel blockage or have narrowing of the stomach or intestines. known hypersensitivity to hydrocodone or acetaminophen, or any ingredient in Hydrocodone Bitartrate and Acetaminophen Tablets Before taking Hydrocodone Bitartrate and Acetaminophen Tablets , tell your healthcare provider if you have a history of: head injury, seizures liver, kidney, thyroid problems problems urinating pancreas or gallbladder problems abuse of street or prescription drugs, alcohol addiction, opioid overdose,or mental health problems. Tell your healthcare provider if you are: pregnant or planning to become pregnant. Prolonged use of Hydrocodone Bitartrate and Acetaminophen Tablets during pregnancy can cause withdrawal symptoms in your newborn baby that could be life-threatening if not recognized and treated. Breast feeding. Hydrocodone bitartrate and acetaminophen pass into breast milk and may harm your baby. living in a household where there are small children or someone who has abused street or prescription drugs taking prescription or over-the-counter medicines, vitamins, or herbal supplements. Taking Hydrocodone Bitartrate and Acetaminophen Tablets with certain other medicines can cause serious side effects that could lead to death. When taking Hydrocodone Bitartrate and Acetaminophen Tablets : Do not change your dose. Take Hydrocodone Bitartrate and Acetaminophen Tablets exactly as prescribed by your healthcare provider. Use the lowest dose possible for the shortest time needed. Take your prescribed dose every four to six hours as needed for pain. Do not take more than your prescribed dose. If you miss a dose, take your next dose at your usual time. Call your healthcare provider if the dose you are taking does not control your pain. If you have been taking Hydrocodone Bitartrate and Acetaminophen Tablets regularly, do not stop taking Hydrocodone Bitartrate and Acetaminophen Tablets without talking to your healthcare provider. Dispose of expired, unwanted, or unused Hydrocodone Bitartrate and Acetaminophen Tablets by promptly flushing down the toilet, if a drug take-back option is not readily available. Visit www.fda.gov/drugdisposal for additional information on disposal of unused medicines. While taking Hydrocodone Bitartrate and Acetaminophen Tablets DO NOT: Drive or operate heavy machinery, until you know how Hydrocodone Bitartrate and Acetaminophen Tablets affect you. Hydrocodone Bitartrate and Acetaminophen Tablets can make you sleepy, dizzy, or lightheaded. Drink alcohol or use prescription or over-the-counter medicines that contain alcohol. Using products containing alcohol during treatment with Hydrocodone Bitartrate and Acetaminophen Tablets may cause you to overdose and die. The possible side effects of Hydrocodone Bitartrate and Acetaminophen Tablets : Constipation, nausea, sleepiness, vomiting, tiredness, headache, dizziness, abdominal pain. Call your healthcare provider if you have any of these symptoms and they are severe. Get emergency medical help or call 911 right away if you have: trouble breathing, shortness of breath, fast heartbeat, chest pain, swelling of your face, tongue, or throat, extreme drowsiness, light-headedness when changing positions, feeling faint, agitation, high body temperature, trouble walking, stiff muscles, or mental changes such as confusion. These are not all the possible side effects of Hydrocodone Bitartrate and Acetaminophen Tablets. Call your doctor for medical advice about side effects. You may report side effects to FDA at 1-800-FDA- 1088. For more information go to dailymed.nlm.nih.gov. What are the ingredients in Hydrocodone Bitartrate and Acetaminophen Tablets? Active Ingredients: Hydrocodone Bitartrate and Acetaminophen Inactive ingredients: colloidal silicon dioxide, croscarmellose sodium, crospovidone, magnesium stearate, microcrystalline cellulose, povidone, pregelatinized starch (maize) and stearic acid. This Medication Guide has been approved by the U.S. Food and Drug Administration. Dispense with Medication Guide available at https://www.aurobindousa.com/medication-guides/ Distributed by: Aurobindo Pharma USA, Inc 279 Princeton-Hightstown Road East Windsor, NJ 08520 Revised: 08/2020"
      ],
      "spl_medguide_table": [
        "<table cellspacing=\"0\" cellpadding=\"0\" border=\"0\" width=\"666\"><colgroup><col width=\"100%\"/></colgroup><tbody><tr styleCode=\"Botrule\"><td styleCode=\"Lrule Rrule\" valign=\"top\"><content styleCode=\"bold\">Hydrocodone Bitartrate (</content>hye&quot; droe koe&apos; done bye tar trate<content styleCode=\"bold\">) and Acetaminophen (</content>a seet&quot;a min&apos; oh fen<content styleCode=\"bold\">) Tablets USP, CII</content> </td></tr><tr styleCode=\"Botrule\"><td styleCode=\"Lrule Rrule\" valign=\"top\"><content styleCode=\"bold\"> Hydrocodone Bitartrate and Acetaminophen Tablets are:</content> <list listType=\"unordered\" styleCode=\"disc\"><item>A strong prescription pain medicine that contains an opioid (narcotic) that is used to manage pain severe enough to require an opioid pain medicine, when other pain treatments such as non-opioid pain medicines do not treat your pain well enough or you cannot tolerate them.</item><item>An opioid pain medicine that can put you at risk for overdose and death. Even if you take your dose correctly as prescribed you are at risk for opioid addiction, abuse, and misuse that can lead to death.</item></list>  </td></tr><tr styleCode=\"Botrule\"><td styleCode=\"Lrule Rrule\" valign=\"top\"><content styleCode=\"bold\">Important information about Hydrocodone Bitartrate and Acetaminophen Tablets :</content> <list listType=\"unordered\" styleCode=\"disc\"><item><content styleCode=\"bold\">Get emergency help or call 911 right away if you take too much Hydrocodone Bitartrate and Acetaminophen Tablets (overdose).</content> When you first start taking Hydrocodone Bitartrate and Acetaminophen Tablets, when your dose is changed, or if you take too much (overdose), serious or life-threatening breathing problems that can lead to death may occur. Talk to your healthcare provider about naloxone, a medicine for the emergency treatment of an opioid overdose.</item><item>Taking Hydrocodone Bitartrate and Acetaminophen Tablets with other opioid medicines, benzodiazepines, alcohol, or other central nervous system depressants (including street drugs) can cause severe drowsiness, decreased awareness, breathing problems, coma, and death.</item><item>Never give anyone else your Hydrocodone Bitartrate and Acetaminophen Tablets. They could die from taking it. Selling or giving away Hydrocodone Bitartrate and Acetaminophen Tablets are against the law.</item><item>Store Hydrocodone Bitartrate and Acetaminophen Tablets securely, out of sight and reach of children, and in a location not accessible by others, including visitors to the home.</item></list></td></tr><tr styleCode=\"Botrule\"><td styleCode=\"Lrule Rrule\" valign=\"top\"><content styleCode=\"bold\"> Do not take Hydrocodone Bitartrate and Acetaminophen Tablets if you have:</content> <list listType=\"unordered\" styleCode=\"disc\"><item>severe asthma, trouble breathing, or other lung problems.</item><item>a bowel blockage or have narrowing of the stomach or intestines.</item><item>known hypersensitivity to hydrocodone or acetaminophen, or any ingredient in Hydrocodone Bitartrate and Acetaminophen Tablets</item></list>  </td></tr><tr styleCode=\"Botrule\"><td styleCode=\"Lrule Rrule\" valign=\"top\"><content styleCode=\"bold\">Before taking Hydrocodone Bitartrate and Acetaminophen Tablets , tell your healthcare provider if you have a history of:</content> <list listType=\"unordered\" styleCode=\"disc\"><item>head injury, seizures</item><item>liver, kidney, thyroid problems</item><item>problems urinating</item><item>pancreas or gallbladder problems</item><item>abuse of street or prescription drugs, alcohol addiction, opioid overdose,or mental health problems.<content styleCode=\"bold\"/></item></list><content styleCode=\"bold\"/> <content styleCode=\"bold\">Tell your healthcare provider if you are:</content> <list listType=\"unordered\" styleCode=\"disc\"><item><content styleCode=\"bold\">pregnant or planning to become pregnant.</content> Prolonged use of Hydrocodone Bitartrate and Acetaminophen Tablets during pregnancy can cause withdrawal symptoms in your newborn baby that could be life-threatening if not recognized and treated.</item><item><content styleCode=\"bold\">Breast feeding.</content> Hydrocodone bitartrate and acetaminophen pass into breast milk and may harm your baby.</item><item>living in a household where there are small children or someone who has abused street or prescription drugs</item><item>taking prescription or over-the-counter medicines, vitamins, or herbal supplements. Taking Hydrocodone Bitartrate and Acetaminophen Tablets with certain other medicines can cause serious side effects that could lead to death.</item></list></td></tr><tr styleCode=\"Botrule\"><td styleCode=\"Lrule Rrule\" valign=\"top\"><content styleCode=\"bold\"> When taking Hydrocodone Bitartrate and Acetaminophen Tablets :</content>   <list listType=\"unordered\" styleCode=\"disc\"><item>Do not change your dose. Take Hydrocodone Bitartrate and Acetaminophen Tablets exactly as prescribed by your healthcare provider. Use the lowest dose possible for the shortest time needed.</item><item>Take your prescribed dose every four to six hours as needed for pain.</item><item>Do not take more than your prescribed dose. If you miss a dose, take your next dose at your usual time.</item><item>Call your healthcare provider if the dose you are taking does not control your pain.</item><item>If you have been taking Hydrocodone Bitartrate and Acetaminophen Tablets regularly, do not stop taking Hydrocodone Bitartrate and Acetaminophen Tablets without talking to your healthcare provider.</item><item> Dispose of expired, unwanted, or unused Hydrocodone Bitartrate and Acetaminophen Tablets by promptly flushing down the toilet, if a drug take-back option is not readily available. Visit <content styleCode=\"bold\"><content styleCode=\"underline\">www.fda.gov/drugdisposal</content></content> for additional information on disposal of unused medicines.</item></list></td></tr><tr styleCode=\"Botrule\"><td styleCode=\"Lrule Rrule\" valign=\"top\"><content styleCode=\"bold\">While taking Hydrocodone Bitartrate and Acetaminophen Tablets DO NOT:</content> <list listType=\"unordered\" styleCode=\"disc\"><item>Drive or operate heavy machinery, until you know how Hydrocodone Bitartrate and Acetaminophen Tablets affect you. Hydrocodone Bitartrate and Acetaminophen Tablets can make you sleepy, dizzy, or lightheaded.</item><item>Drink alcohol or use prescription or over-the-counter medicines that contain alcohol. Using products containing alcohol during treatment with Hydrocodone Bitartrate and Acetaminophen Tablets may cause you to overdose and die.</item></list></td></tr><tr styleCode=\"Botrule\"><td styleCode=\"Lrule Rrule\" valign=\"top\"><content styleCode=\"bold\">The possible side effects of Hydrocodone Bitartrate and Acetaminophen Tablets :</content> <list listType=\"unordered\" styleCode=\"disc\"><item>Constipation, nausea, sleepiness, vomiting, tiredness, headache, dizziness, abdominal pain. Call your healthcare provider if you have any of these symptoms and they are severe.</item></list></td></tr><tr><td styleCode=\"Lrule Rrule\" align=\"justify\" valign=\"top\"><content styleCode=\"bold\">Get emergency medical help or call 911 right away if you have:</content> <list listType=\"unordered\" styleCode=\"disc\"><item>trouble breathing, shortness of breath, fast heartbeat, chest pain, swelling of your face, tongue, or throat, extreme drowsiness, light-headedness when changing positions, feeling faint, agitation, high body temperature, trouble walking, stiff muscles, or mental changes such as confusion.</item></list>These are not all the possible side effects of Hydrocodone Bitartrate and Acetaminophen Tablets. Call your doctor for medical advice about side effects. You may report side effects to FDA at 1-800-FDA- 1088. <content styleCode=\"bold\">For more information go to dailymed.nlm.nih.gov.</content> <content styleCode=\"bold\"> What are the ingredients in Hydrocodone Bitartrate and Acetaminophen Tablets?</content> <content styleCode=\"bold\"> Active Ingredients: Hydrocodone Bitartrate and Acetaminophen</content> <content styleCode=\"bold\">Inactive ingredients: </content><content styleCode=\"bold\">colloidal silicon dioxide, croscarmellose sodium, crospovidone, magnesium stearate, microcrystalline cellulose, povidone, pregelatinized starch (maize) and stearic acid.</content> </td></tr></tbody></table>"
      ],
      "package_label_principal_display_panel": [
        "PACKAGE LABEL-PRINCIPAL DISPLAY PANEL \u2013 5 mg/300 mg (100 Tablets Bottle) NDC 13107-211-01 Hydrocodone Bitartrate and Acetaminophen Tablets USP CII 5 mg/300 mg Rx only 100 Tablets AUROBINDO hydrocodone5mg100ct",
        "PACKAGE LABEL-PRINCIPAL DISPLAY PANEL \u2013 7.5 mg/300 mg (100 Tablets Bottle) NDC 13107-212-01 Hydrocodone Bitartrate and Acetaminophen Tablets USP CII 7.5 mg/300 mg Rx only 100 Tablets AUROBINDO hydrocodone7.5mg100ct",
        "PACKAGE LABEL-PRINCIPAL DISPLAY PANEL \u2013 10 mg/300 mg (100 Tablets Bottle) NDC 13107-213-01 Hydrocodone Bitartrate and Acetaminophen Tablets USP CII 10 mg/300 mg Rx only 100 Tablets AUROBINDO hydrocodone10mg100ct"
      ],
      "set_id": "392a846e-7325-462d-b19b-8afda44c5cb0",
      "id": "c41e5c06-bede-4ccc-b5d5-ebed1867077c",
      "effective_time": "20200902",
      "version": "6",
      "application_number": [
        "ANDA207709"
      ],
      "brand_name": [
        "Hydrocodone Bitartrate and Acetaminophen"
      ],
      "generic_name": [
        "HYDROCODONE BITARTRATE AND ACETAMINOPHEN"
      ],
      "manufacturer_name": [
        "Aurolife Pharma, LLC"
      ],
      "product_ndc": [
        "13107-211",
        "13107-212",
        "13107-213"
      ],
      "product_type": [
        "HUMAN PRESCRIPTION DRUG"
      ],
      "route": [
        "ORAL"
      ],
      "substance_name": [
        "ACETAMINOPHEN",
        "HYDROCODONE BITARTRATE"
      ],
      "rxcui": [
        "856980",
        "856987",
        "856992"
      ],
      "spl_id": [
        "c41e5c06-bede-4ccc-b5d5-ebed1867077c"
      ],
      "spl_set_id": [
        "392a846e-7325-462d-b19b-8afda44c5cb0"
      ],
      "package_ndc": [
        "13107-211-30",
        "13107-211-01",
        "13107-211-05",
        "13107-211-99",
        "13107-212-30",
        "13107-212-01",
        "13107-212-05",
        "13107-212-99",
        "13107-213-30",
        "13107-213-01",
        "13107-213-05",
        "13107-213-99"
      ],
      "is_original_packager": [
        "true"
      ],
      "upc": [
        "0313107213014",
        "0313107211010"
      ],
      "unii": [
        "NO70W886KK",
        "362O9ITL9D"
      ],
      "last_update" : "2024-06-18"
    }



In [ ]:
# Expresión regular precompilada para eliminar etiquetas HTML
cl = re.compile('<.*?>')

# Funcion para limpiar los datos y quitar caracteres innesarios como los html
def remove_tags(text):
    #text = text.decode('utf-8')
    return re.sub(cl, '', text)

# Funcion para convertir un objeto tipo lista a un objeto tipo string  
def s_encode(text):
    if(text is not None):
        if isinstance(text, list):
            text = " ".join(map(str, text))
        else:
            text = str(text)
    return text
    #return str(text).encode('utf-8')



In [ ]:
# Dataframe de prueba
df_nuevo = pd.json_normalize(batch)
df_nuevo['Atest'] = np.nan
df_nuevo = df_nuevo.apply(lambda x: x.apply(s_encode) if(x is not None) else x, axis=0)
df_nuevo = df_nuevo.apply(lambda x: x.apply(remove_tags)if(x is not None) else x, axis=0)
print(df_nuevo['package_label_principal_display_panel'][0])
print()
print(df_nuevo['package_ndc'][0])
print()
print(df_nuevo['is_original_packager'][0])
print()
print(df_nuevo['dosage_and_administration_table'][0])
print()
print(df_nuevo['Atest'][0])


In [43]:
!ls -lrth $DATASET_FOLDER
!rm -R $DATASET_FOLDER/*.json

total 8.5G
-rw-r--r-- 1 root root 603M Aug  7 02:49 drug-label-0001-of-0012.json
-rw-r--r-- 1 root root 609M Aug  7 02:49 drug-label-0002-of-0012.json
-rw-r--r-- 1 root root 672M Aug  7 02:49 drug-label-0003-of-0012.json
-rw-r--r-- 1 root root 667M Aug  7 02:50 drug-label-0004-of-0012.json
-rw-r--r-- 1 root root 646M Aug  7 02:50 drug-label-0005-of-0012.json
-rw-r--r-- 1 root root 645M Aug  7 02:50 drug-label-0006-of-0012.json
-rw-r--r-- 1 root root 648M Aug  7 02:50 drug-label-0007-of-0012.json
-rw-r--r-- 1 root root 659M Aug  7 02:50 drug-label-0008-of-0012.json
-rw-r--r-- 1 root root 579M Aug  7 02:50 drug-label-0009-of-0012.json
-rw-r--r-- 1 root root 586M Aug  7 02:50 drug-label-0010-of-0012.json
-rw-r--r-- 1 root root 653M Aug  7 02:51 drug-label-0011-of-0012.json
-rw-r--r-- 1 root root 359M Aug  7 02:51 drug-label-0012-of-0012.json
-rw-r--r-- 1 root root 1.3G Aug  7 02:58 drugs_data.parquet
